# Download Permafrost Data

The [Global Terrestrial Network for Permafrost - Database](http://www.gtnpdatabase.org/) contains a repository on data from permafrost sites. Data for both
 * Thaw Depths (aka Active Layer Thickness)
 * Temperatures
are available.

Unfortunately, the site focuses on making the data available individually involving a lot of user interaction. For analyses at scale, this proves inefficient. This notebook downloads the data locally so you can analyse the data yourself.

We are trying to build a simplified version of the GTN-P data with as few tables as possible. With the source using normalized tables, consistency of GTN-P is generally good, so, for the sake of data science ease of use, we use "fat" tables.

![Data Model](Data_Model.png)


## Packages

There are a few packages which may not be part of a default installation. Also, some may require an installation at the OS level outside the realm of pipenv, conda, or pip.
* [geopandas](https://geopandas.org/) is required to save data with geospatial information
* [PostgreSQL](https://www.postgresql.org/) and its Geospatial Information System extension [PostGIS](https://postgis.net/install/) are required locally. We will try to make a docker package available. Note that PostGIS needs to be enabled per-database. For Mac users, consider [Postgret.app](https://postgresapp.com/), for Windows users, consider [PostgreSQL Portable](https://github.com/garethflowers/postgresql-portable)
* [pycountry](https://pypi.org/project/pycountry/) is used to standardise country names and look up their ISO codes
* [Selenium](https://pypi.org/project/selenium/) is required to load pages that use JavaScript to build their content. This generally requires some installation on the OS as Selenium needs a working web browser to render these pages.
* [tqdm](https://github.com/tqdm/tqdm) for progress bars (although we could also use ipywidgets progress bar, but tqdm also measures and displays time per iteration and often ETA)
* We also use sqlalchemy, pandas, numpy, and scipy. To avoid data conversion difficulties, we store local data files as binary files, preference being parquet, so pyarrow or fastparquet may be required.

In [1]:
import io
import pandas as pd
import geopandas as gpd
import shapely
from bs4 import BeautifulSoup
import bs4
import requests
import zipfile
from pathlib import Path
import time
from tqdm import tqdm
import numpy as np
import pycountry
import glob
import os
import glob
import re
from selenium import webdriver
from sqlalchemy import create_engine, text
import sqlalchemy
import psycopg2
from tqdm import tqdm
import json
import datetime
import socket
import urllib
import sys
from scipy.interpolate import interp1d

## Web Crawling using Selenium

The web page is dynamically generated using JavaScript. This makes access the content slightly more challenging. Selenium is a toolset that is created to allow for automated UI testing, but it is also used to crawl websites. 

Depending on your jupyter setup, selenium may be very tricky, or impossible to install, as it actually requires a functioning web browser to be installed on the machine that hosts the jupyter evironment. This is very often not the case for servers or cloud based installations. Should this be the case:
* Open [http://www.gtnpdatabase.org/activelayers](http://www.gtnpdatabase.org/activelayers) in a browser on your workstation
* Try to File->Save the webpage locally
* There will be a hook for you to copy and paste the file further down

In [2]:
options = webdriver.ChromeOptions() # https://stackoverflow.com/questions/50642308
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome(options=options)

If you cannot get Selenium installed or working, open [http://www.gtnpdatabase.org/activelayers](http://www.gtnpdatabase.org/activelayers) and save it as `./sun/activelayers/index.html`. Same for [http://www.gtnpdatabase.org/boreholes](http://www.gtnpdatabase.org/boreholes) and `./sun/boreholes/index.html`.

In [3]:
Path("./sun/activelayers").mkdir(exist_ok=True,parents=True)

wd.get("http://www.gtnpdatabase.org/activelayers")
print("waiting 10 seconds for the JavaScript to complete",end=".")
time.sleep(10) # wait for javascript to finish
print("...done, next page")
activelayers_page = wd.page_source

wd.get("http://www.gtnpdatabase.org/boreholes")
print("waiting 10 seconds for the JavaScript to complete",end=".")
time.sleep(10) # wait for javascript to finish
print("...finished")
boreholes_temps_page = wd.page_source

waiting 10 seconds for the JavaScript to complete....done, next page
waiting 10 seconds for the JavaScript to complete....finished


Unfortunately, the usual way to parse the table using pandas´ `read_html` results in a missing entry, Station _Da Xi Gou  (Glacier Station)_  is not always added. We will use that approach further down in this notebook.

In [4]:
df = pd.read_html(boreholes_temps_page)[0] # little known fact, pandas parses tables quite well, note it returns an array
df.columns = [c.lower() for c in df.columns] 
df[df["gtn-p"]=="CN9"]

,name,site,country,gtn-p,vegetation,permafrost,elevation,depth,data,select


### Helper Function to Log Data Issues

As we walk across the data, we will encounter deficiencies. We will set up a log database so that, if possible, deficiencies can be rectified.

In [5]:
def static_vars(**kwargs):
    def decorate(func):
        for k in kwargs:
            setattr(func, k, kwargs[k])
        return func
    return decorate

@static_vars(log=[])
def log_finding(finding):
    finding["datetime_date"] = "{:%Y-%m-%d %H:%M:%S}".format(datetime.datetime.now())
    finding["machine"] = socket.gethostname()
    log_finding.log.append(finding)
    with open("findings.json","w+t") as logfile:
        json.dump(log_finding.log,logfile)

## Database connection

If you plan to put code of yours onto public platforms such as github or gitlab, never, ever, add credentials to your source code. In fact, never ever, even if you have no such plans. We read the credentials and database server from a file names .env, in this repo, copy .env.example and edit the content so it suits your needs. We need a string in the form 

```
postgresql://<user>:<password>@<hostname>/<database>
```

in that file. Often, this is `postgresql://postgres@localhost/postgres` by default.

In [6]:
with open(".env","rt") as idfile:
    connect_string = idfile.read().strip()
conn = create_engine(connect_string)

## Web Page parsing using BeautifulSoup

The download links use an id of the site in their URL. The id field, however, is not visible on the web page, but is in the hyperlink `href` field of the table. These links are contained in c table cell (`<td>`) with an attribute `data-th="Name"`.

The html code looks like

```
<td data-th="Name"><a class="ng-binding" href="/activelayers/view/195/">Allakaket</a></td>
```

and we are after the 195, which is the last forward-slash separated field of the URL. Also, as we already use BeautifulSoup, we take this approach to populate the data frame.

### Activelayers

In [7]:
soup = BeautifulSoup(io.StringIO(activelayers_page))

entries = []
for tr in soup.findAll("tr"):
    entry = {}
    for td in tr.findAll("td"):
        if td["data-th"] == "Gtn-P":
            entry["code"] = td.text
        else:
            entry[td["data-th"].lower()] = td.text
        if td["data-th"] == "Name":
            entry["subsite_id"] = td.contents[0]["href"].split("/")[-2]
            entry["view_url"] = td.contents[0]["href"]
        elif td["data-th"] == "Site":
            entry["site_id"] = td.contents[0]["href"].split("/")[-2]
            entry["site_url"] = td.contents[0]["href"]
    if len(entry)>0:
        entries.append(entry)
dfActiveLayersCatalogue = pd.DataFrame(entries).rename(columns={"name":"subsite_name",
                                                               "site":"site_name"})
dfActiveLayersCatalogue.elevation = pd.to_numeric(dfActiveLayersCatalogue.elevation)
print("{} Entries loaded".format(len(dfActiveLayersCatalogue)))

253 Entries loaded


Add standard country codes and names using `pycountry`.

In [8]:
dfActiveLayersCatalogue["iso2"] = ""
dfActiveLayersCatalogue["iso3"] = ""
dfActiveLayersCatalogue["country_official_name"] = ""
for i,r in dfActiveLayersCatalogue.iterrows():
    info = pycountry.countries.search_fuzzy(r.country)
    dfActiveLayersCatalogue.at[i,"iso2"] = info[0].alpha_2
    dfActiveLayersCatalogue.at[i,"iso3"] = info[0].alpha_3
    if "country_official_name" in info[0].__dict__.keys():
        dfActiveLayersCatalogue.at[i,"country_official_name"] = info[0].country_official_name
    else:
        dfActiveLayersCatalogue.at[i,"country_official_name"] = info[0].name
dfActiveLayersCatalogue["subsite_type"] = "activelayer"
dfActiveLayersCatalogue.head()

,subsite_name,subsite_id,view_url,site_name,site_id,site_url,country,type,code,vegetation,permafrost,elevation,data,select,iso2,iso3,country_official_name,subsite_type
0,56 Mile,227,/activelayers/view/227/,Franklin Bluff,16,/sites/view/16/,United States,Grid,U31 A,Tundra,Continuous,114.0,No,,US,USA,United States,activelayer
1,Abisko,42,/activelayers/view/42/,Abisko,6,/sites/view/6/,Sweden,Grid,S2,Forest Tundra,Discontinuous,507.0,No,,SE,SWE,Sweden,activelayer
2,Akhmelo Channel,88,/activelayers/view/88/,Cherskii,8,/sites/view/8/,Russia,Grid,R17,Forest Tundra,Continuous,5.0,Yes,,RU,RUS,Russian Federation,activelayer
3,Alazeya River,82,/activelayers/view/82/,Cherskii,8,/sites/view/8/,Russia,Grid,R22,Shrub Tundra,Continuous,60.0,Yes,,RU,RUS,Russian Federation,activelayer
4,Alexandria Fiord,27,/activelayers/view/27/,Alexandria Fiord,329,/sites/view/329/,Canada,Grid,C1,Tundra,Continuous,0.0,Yes,,CA,CAN,Canada,activelayer


### Temperatures

In [9]:
soup = BeautifulSoup(io.StringIO(boreholes_temps_page), 'lxml')

entries = []
for tr in soup.findAll("tr"):
    entry = {}
    for td in tr.findAll("td"):
        if td["data-th"] == "Gtn-P":
            entry["code"] = td.text
        else:
            entry[td["data-th"].lower()] = td.text
        if td["data-th"] == "Name":
            entry["subsite_id"] = td.contents[0]["href"].split("/")[-2]
            entry["view_url"] = td.contents[0]["href"]
        elif td["data-th"] == "Site":
            entry["site_id"] = td.contents[0]["href"].split("/")[-2]
            entry["site_url"] = td.contents[0]["href"]
    if len(entry)>0:
        entries.append(entry)
dfBoreholeTempsCatalogue = pd.DataFrame(entries).rename(columns={"name":"subsite_name",
                                                                 "site":"site_name"})
dfBoreholeTempsCatalogue.elevation = pd.to_numeric(dfBoreholeTempsCatalogue.elevation)
print("{} Entries loaded".format(len(dfBoreholeTempsCatalogue)))

1380 Entries loaded


There is a typo in the country names, which we want to log.

In [10]:
dfBoreholeTempsCatalogue["iso2"] = ""
dfBoreholeTempsCatalogue["iso3"] = ""
dfBoreholeTempsCatalogue["country_official_name"] = ""
for i,r in dfBoreholeTempsCatalogue.iterrows():
    if r.country.lower() == "kyrgystan":
        log_finding({"topic":"borehole temperatures",
                    "site_id":r.site_id,
                    "subsite_id":r.subsite_id,
                    "site_url":r.site_url,
                    "site_name":r.site_name,
                    "diagnosis":"Country spelling incorrect, '{}' is spelled 'Kyrgyzstan'".format(r.country),
                    "fix":"changed to 'Kyrgyzstan'",
                    "needs_attention":False})
        info = pycountry.countries.search_fuzzy("Kyrgyzstan")
    else:
        info = pycountry.countries.search_fuzzy(r.country)
    dfBoreholeTempsCatalogue.at[i,"iso2"] = info[0].alpha_2
    dfBoreholeTempsCatalogue.at[i,"iso3"] = info[0].alpha_3
    if "country_official_name" in info[0].__dict__.keys():
        dfBoreholeTempsCatalogue.at[i,"country_official_name"] = info[0].country_official_name
    else:
        dfBoreholeTempsCatalogue.at[i,"country_official_name"] = info[0].name
        
dfBoreholeTempsCatalogue["subsite_type"] = "temperatures"
dfBoreholeTempsCatalogue.head()

,subsite_name,subsite_id,view_url,site_name,site_id,site_url,country,code,vegetation,permafrost,elevation,depth,data,select,iso2,iso3,country_official_name,subsite_type
0,0 (Deputatsky),1007,/boreholes/view/1007/,Deputatskiy,331,/sites/view/331/,Russia,RU 118,Shrub Tundra,Continuous,462.0,88.00,No,,RU,RUS,Russian Federation,temperatures
1,01TC1,1744,/boreholes/view/1744/,Yukon,73,/sites/view/73/,Canada,CA 196,Grassland,Continuous,18.0,8.00,No,,CA,CAN,Canada,temperatures
2,01TC2,1745,/boreholes/view/1745/,Yukon,73,/sites/view/73/,Canada,CA 197,Grassland,Continuous,95.0,10.00,No,,CA,CAN,Canada,temperatures
3,03TC1,1743,/boreholes/view/1743/,Yukon,73,/sites/view/73/,Canada,CA 195,Grassland,Discontinuous,3.0,6.00,No,,CA,CAN,Canada,temperatures
4,08 (Deputatsky),1008,/boreholes/view/1008/,Deputatskiy,331,/sites/view/331/,Russia,RU 119,Forest Tundra,Continuous,473.0,96.00,No,,RU,RUS,Russian Federation,temperatures


### Combine Borehole Depth and Temperatures Catalogues into one

There is historic reasons these were created separately. Our aim is to assess all permafrost data, so we want one catalogue.

In [11]:
dfBoreholeTempsCatalogue["type"] = "N/A" # pad a missing field which is only present in the borhole data
dfSites = dfActiveLayersCatalogue.append(dfBoreholeTempsCatalogue).reset_index()
# convert Yes/No text into boolean
dfSites["have_data"] = False
idx = dfSites[dfSites["data"] == "Yes"].index
dfSites.at[idx,"have_data"] = True
del dfSites["data"]
del dfSites["index"]
#dfSites.rename(columns={"name":"subsite_name","site":"site_name"},inplace=True)
dfSites

,subsite_name,subsite_id,view_url,site_name,site_id,site_url,country,type,code,vegetation,permafrost,elevation,select,iso2,iso3,country_official_name,subsite_type,depth,have_data
0,56 Mile,227,/activelayers/view/227/,Franklin Bluff,16,/sites/view/16/,United States,Grid,U31 A,Tundra,Continuous,114.0,,US,USA,United States,activelayer,NaN,False
1,Abisko,42,/activelayers/view/42/,Abisko,6,/sites/view/6/,Sweden,Grid,S2,Forest Tundra,Discontinuous,507.0,,SE,SWE,Sweden,activelayer,NaN,False
2,Akhmelo Channel,88,/activelayers/view/88/,Cherskii,8,/sites/view/8/,Russia,Grid,R17,Forest Tundra,Continuous,5.0,,RU,RUS,Russian Federation,activelayer,NaN,True
3,Alazeya River,82,/activelayers/view/82/,Cherskii,8,/sites/view/8/,Russia,Grid,R22,Shrub Tundra,Continuous,60.0,,RU,RUS,Russian Federation,activelayer,NaN,True
4,Alexandria Fiord,27,/activelayers/view/27/,Alexandria Fiord,329,/sites/view/329/,Canada,Grid,C1,Tundra,Continuous,0.0,,CA,CAN,Canada,activelayer,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628,Zima railst,1677,/boreholes/view/1677/,Irkutsk,369,/sites/view/369/,Russia,N/A,,Other,Isolated patches,456.0,,RU,RUS,Russian Federation,temperatures,3.00,True
1629,Zugspitze 01/07 (W),859,/boreholes/view/859/,Zugspitze,263,/sites/view/263/,Germany,N/A,DE 01,Other,Isolated Patches,2922.0,,DE,DEU,Germany,temperatures,44.00,False
1630,Zugspitze 02/07 (E),858,/boreholes/view/858/,Zugspitze,263,/sites/view/263/,Germany,N/A,DE 02,Other,Isolated Patches,2922.0,,DE,DEU,Germany,temperatures,58.00,False
1631,Zugspitze tunnel,1798,/boreholes/view/1798/,Zugspitze,263,/sites/view/263/,Germany,N/A,,No Vegetation,Mountain,2785.0,,DE,DEU,Germany,temperatures,50.00,False


Unfortunately, subsite numerical IDs are not unique as the website had been set up to discriminate between active layers and temperature measurements. So, we need to change the subsite ID to also contain an indicator for the subsite type, we chose _a_ for _activelayer, and _t_ for _temperatures_.

In [12]:
dfTmp = pd.DataFrame(dfSites.groupby("subsite_id").size())
dfTmp[dfTmp[0]>1]

,0
subsite_id,
100,2
101,2
102,2
103,2
104,2
...,...
95,2
96,2
97,2


In [13]:
dfSites["subsite_id"] = dfSites["subsite_type"].str.slice(0,1)+dfSites["subsite_id"].astype(str).str.zfill(4)
dfTmp = pd.DataFrame(dfSites.groupby("subsite_id").size())
dfTmp[dfTmp[0]>1]

,0
subsite_id,


In [14]:
Path("./sun/catalogue").mkdir(exist_ok=True,parents=True)
dfSites.to_parquet("./sun/catalogue/sites.catalogue.parquet")

In [15]:
#dfSites = pd.read_parquet("./sun/catalogue/sites.catalogue.parquet")

## Download Site Metadata

Each site has one landing page which describes more about its location and some subsites. In particular, we are after Observation Type, Area, Latitude/Longitude, which is not part of the previous dataset.

Note that we have a hybrid of site and subsite data. In particular, Latitude and Longitude are site data, while name is the subsite name.

As all data we want are inside html tables, we use pandas' `read_html`, which returns a list of dataframes.

In [16]:
nok = []
alldata = []

site_urls = dfSites.site_url.unique()

for site_url in tqdm(site_urls):
    Path("./sun"+site_url).mkdir(exist_ok=True,parents=True)
    if not os.path.exists("./sun"+site_url+"index.html"):
        r = requests.get("http://www.gtnpdatabase.org{}".format(site_url))
        if r.ok:
            with open("./sun"+site_url+"index.html","w+") as outfile:
                outfile.write(r.text)
        else:
            nok.append("http://www.gtnpdatabase.org{}".format(site_url))
            continue
    dfGeneral,dfSiteCentre,dfBBox = pd.read_html("./sun"+site_url+"index.html")
    alldata.append({"site_id":site_url.split("/")[-2],
                   "observation_type":dfGeneral.iloc[1].values[1],
                  "area":pd.to_numeric(dfGeneral.iloc[2].values[1].replace(",","").split(" ")[0]),
                  "site_lon":pd.to_numeric(dfSiteCentre.iloc[0].values[1].replace("°","")),
                  "site_lat":pd.to_numeric(dfSiteCentre.iloc[1].values[1].replace("°","")),
                  "bbox_lat_max":pd.to_numeric(dfBBox.iloc[0].values[1].replace("°","").split(":")[1]),
                  "bbox_lat_min":pd.to_numeric(dfBBox.iloc[2].values[1].replace("°","").split(":")[1]),
                  "bbox_lon_min":pd.to_numeric(dfBBox.iloc[1].values[0].replace("°","").split(":")[1]),
                  "bbox_lon_max":pd.to_numeric(dfBBox.iloc[1].values[2].replace("°","").split(":")[1])})
    time.sleep(.25)

if len(nok)>0:
    print("Warning, had problems downloading {}".format(nok))
dfSiteInfo = pd.DataFrame(alldata)
dfSiteInfo.observation_type = dfSiteInfo.observation_type.replace(np.nan, '', regex=True)

100%|██████████| 352/352 [01:32<00:00,  3.80it/s]


In [17]:
dfSites = dfSites.merge(dfSiteInfo,on="site_id")
Path("./mercury/catalogue").mkdir(exist_ok=True,parents=True)
dfSites.to_parquet("./mercury/catalogue/sites.catalogue.parquet")

In [18]:
dfSites = pd.read_parquet("./mercury/catalogue/sites.catalogue.parquet")

In [19]:
gdfSites = gpd.GeoDataFrame(dfSites, geometry=gpd.points_from_xy(dfSites.site_lon,dfSites.site_lat)).set_crs(epsg=4326)
for c in ["site_id","elevation","depth","area","site_lat","site_lon","bbox_lat_max","bbox_lat_min","bbox_lon_min","bbox_lon_max"]:
    gdfSites[c] = pd.to_numeric(gdfSites[c])
del gdfSites["select"]

In [21]:
stmt = """CREATE TABLE t_permafrost_sites (
	"subsite_name" varchar(100), 
	"subsite_id" varchar(5), 
	"view_url" varchar(100), 
	"site_name" varchar(100), 
	"site_id" integer, 
	"site_url" varchar(100), 
	"country" varchar(40), 
	"type" varchar(20), 
	"code" varchar(20), 
	"vegetation" varchar(50), 
	"permafrost" varchar(50), 
	"elevation" double precision, 
	"iso2" varchar(2), 
	"iso3" varchar(3), 
	"country_official_name" varchar(80), 
	"subsite_type" varchar(20), 
	"depth" double precision, 
	"have_data" boolean, 
	"observation_type" varchar(20), 
	"area" double precision, 
	"site_lon" double precision, 
	"site_lat" double precision, 
	"bbox_lat_max" double precision, 
	"bbox_lat_min" double precision, 
	"bbox_lon_min" double precision, 
	"bbox_lon_max" double precision, 
	"geometry" geometry
);"""
conn.execute(text(stmt).execution_options(autocommit=True))

In [22]:
gdfSites.to_postgis("t_permafrost_sites",con=conn,dtype={"subsite_name":sqlalchemy.types.VARCHAR(100),
                                                         "subsite_id":sqlalchemy.types.VARCHAR(5),
                                                         "view_url":sqlalchemy.types.VARCHAR(100),
                                                         "site_name":sqlalchemy.types.VARCHAR(100),
                                                         "site_id":sqlalchemy.types.INT,
                                                         "site_url":sqlalchemy.types.VARCHAR(100),
                                                         "country":sqlalchemy.types.VARCHAR(40),
                                                         "type":sqlalchemy.types.VARCHAR(20),
                                                         "code":sqlalchemy.types.VARCHAR(20),
                                                         "vegetation":sqlalchemy.types.VARCHAR(50),
                                                         "permafrost":sqlalchemy.types.VARCHAR(50),
                                                         "iso2":sqlalchemy.types.VARCHAR(2),
                                                         "iso3":sqlalchemy.types.VARCHAR(3),
                                                         "country_official_name":sqlalchemy.types.VARCHAR(80),
                                                         "subsite_type":sqlalchemy.types.VARCHAR(20),
                                                         "observation_type":sqlalchemy.types.VARCHAR(20),
                                                        },if_exists="replace")

In [23]:
comments = {'subsite_name': 'Name of the subsite (usually the hole itself)',
 'subsite_id': 'ID of subsite as generated by gtnpdatabase.org',
 'view_url': 'path portion of subsite webpage',
 'site_name': 'Name of the site (usually a collection of boreholes or a place name)',
 'site_id': 'numerical ID of site as generated by gtnpdatabase.org',
 'site_url': 'path portion of site webpage',
 'country': 'country name where site is located in',
 'type': 'enumeration describing borehole layouts',
 'code': 'a TBD code identifying some (sub)sites',
 'vegetation': 'enumerated class of vegetation surrounding subsite',
 'permafrost': 'enumerated class of permafrost surrounding subsite',
 'elevation': 'elevation of subsite in meters',
 'iso2': 'ISO 3166 ALPHA-2 country code where site is located in',
 'iso3': 'ISO 3166 ALPHA-3 country code where site is located in',
 'country_official_name': 'An official name of the country',
 'subsite_type': 'one of activelayer or temperatures',
 'depth': 'depth of borehole in meters',
 'have_data': 'flag indicating if time series data are available',
 'observation_type': 'One of TSP,CALM,or both indicating TBD',
 'area': 'area of site im km2',
 'site_lon': 'nominal longitude of site',
 'site_lat': 'nominal latitude of site',
 'bbox_lat_max': 'max latitude of site extension',
 'bbox_lat_min': 'min latitude of site extension',
 'bbox_lon_min': 'max longitude of site extension',
 'bbox_lon_max': 'min longitude of site extension',
 'geometry': 'GIS geometry of site longitude/latitude'}

In [24]:
# https://stackoverflow.com/questions/60210704/how-can-i-insert-column-comments-in-postgresql-via-python
# 
table = "t_permafrost_sites"
for column,comment in comments.items():
    stmt = f"COMMENT ON COLUMN {table}.{column} is '{comment}';"
    conn.execute(text(stmt).execution_options(autocommit=True))

In [25]:
#gdfSites = gpd.read_postgis("SELECT * FROM t_permafrost_sites",con=conn,geom_col="geometry")

## Subsite Metadata

Similar to sites, which are a collection of one or more subsites (which actually resemble the borholes, or borehole arrays), subsites have their own metadata site. Metadata there varies between activelayer and temperatures subsites, also, some tables are not present in the html code if no data are associated with them.

In [29]:
def get_dates_from_soup(soup):
    dates = {}
    for e in soup.find("fieldset").findAll("small"):
        fields = e.text.split(":")
        key = fields[0].lower()
        value = pd.to_datetime(fields[1])
        dates[key] = value
    return dates

def get_explanation_from_soup(soup):
    text = []
    for p in soup.find("div",{"class":"leftCol"}).findAll("p",{"class":""}):
        if p.find("strong"):
            header = p.find("strong")
            text.append("# "+header.text)
        elif p.find("a"):
            link = p.find("a",href=True)
            text.append("[{}]({})".format(link.text,link["href"]))
        else:
            text.append(p.text.strip())
    return ("\n\n".join(text))

def get_references_from_soup(soup,site_id,conn,DEFER=True):
    references = []
    entries = []
    el = soup.find('hr')
    while(el):
        el = el.next_sibling
        if isinstance(el, bs4.element.NavigableString):
            if len(el.strip())>0:
                entries.append(el.strip()) 

    for entry in entries:
        try:
            dfExisting = pd.read_sql("SELECT * FROM t_references WHERE reference='{}'".format(
                entry.replace("'","''")),con=conn) # need this as we use a single ' to quote a string
            if len(dfExisting)>0:
                references.extend(dfExisting.doi.values)
                continue
        except:
            print(entry,end="\n")
            pass
        
        url = "https://api.crossref.org/works?query={}".format(urllib.parse.quote(entry))
        if DEFER:
            with open("deferred.queries.txt","a+t") as todolist:
                todolist.write(url+"\n")
            continue
        #r = requests.get(url,headers=headers)
        r = requests.get(url)
        record = {"doi":"","reference":entry,"site_id":site_id}
        if r.ok:
            result = json.loads(r.text)
            item = result["message"]["items"][0]
            if "URL" in item.keys():
                new_record = {"doi":item["URL"],"reference":entry,"result":json.dumps(item)}
        else:
            continue
            
        if "x-rate-limit-limit" in r.headers.keys():
            rate_limit = r.headers["x-rate-limit-limit"]
            rate_interval = pd.to_numeric(r.headers["x-rate-limit-interval"].replace("s",""))
            try:
                sleep = rate_interval/rate_limit
            except:
                sleep = 0.1
        time.sleep(sleep)
        references.append(new_record["doi"])
        xdf = pd.DataFrame(new_record,index=[0])
        xdf.to_sql("t_references",con=conn,dtype={"doi":sqlalchemy.types.VARCHAR(100),
                                   "reference":sqlalchemy.types.VARCHAR(1000),
                                   "result":sqlalchemy.types.VARCHAR(50000)},
                   if_exists="append",index=False)
    return entries,references

In [30]:
keep_columns = {'CALM-Code:':"calm_code", 'Drilling Angle:':"drilling_angle", 
                'Drilling Method:':"drilling_method", 'Elevation:':"subsite_elevation", 
                'GTN-P:':"gtn_p", 'Latitude:':"subsite_lat", 'Longitude:':"subsite_lon",
                'Lithology:':"lithology", 'Morphology:':"morphology", 
                'Responsible Countries:':"responsible_countries",
                'Responsible Person:':"responsible_person"}

In [32]:
nok = []
manual_review = []

adfSubsiteInfo = []

for i,row in tqdm(gdfSites.iterrows()):
    targetfolder = "./sun"+row.view_url
    targetfile = os.path.join(targetfolder,"index.html")
    if os.path.exists(targetfile):
        with open(targetfile) as infile:
            content = infile.read()
    else:
        Path(targetfolder).mkdir(exist_ok=True,parents=True)
        r = requests.get("http://www.gtnpdatabase.org{}".format(row.view_url)) #"/boreholes/view/1097/")) #
        if r.ok:
            content = r.text
        else:
            nok.append(row.view_url)
            continue
        with open(targetfile,"w+") as outfile:
            outfile.write(content)

    soup = BeautifulSoup(content)
    
    df = pd.DataFrame()
    tabs = {0:"general"}
    for infoTable in soup.findAll("table",{"class":"infoTable"}):
        try:
            df = pd.read_html(io.StringIO(str(infoTable)))[0]
        except:
            continue
    dates = get_dates_from_soup(soup)
    dfArray = pd.read_html(io.StringIO(content))

    # we dont try to parse all information...
    #for i in range(len(dfArray)):
    #    dfArray[i]["array_no"] = i

    ddf = pd.DataFrame().append(dfArray)
    ddf = ddf[ddf[0].isin(keep_columns.keys())][[0,1]]
    ddf = ddf.append(pd.DataFrame({0:["subsite_type","created","modified","subsite_id","view_url"],
                                   1:[row.subsite_type,dates["created"],dates["modified"],row.subsite_id,row.view_url]}))

    if ddf.duplicated(subset=[0]).any():
        print("warning")
        break

    if soup.find("div",{"class":"references"}):
        entries,references = get_references_from_soup(soup.find("div",{"class":"references"}),
                                                  row.site_id,conn,DEFER=False)
    else:
        entries = []
        references = []
    
    ddf = ddf.append(pd.DataFrame({0:["entries","references"],
                                   1:[json.dumps(entries),json.dumps(references)]}))
    ddf.index = ddf[0]
    del ddf[0]
    ddf = ddf.transpose()
    adfSubsiteInfo.append(ddf.rename(columns=keep_columns))
    
dfSubsites = pd.DataFrame().append(adfSubsiteInfo)

1633it [12:52,  2.11it/s]


In [33]:
for stmt in ["COMMENT ON COLUMN t_references.doi is 'ISO 26324 identifier and URL for the article';",
             "COMMENT ON COLUMN t_references.reference is 'Original free form text of reference from web page';",
             "COMMENT ON COLUMN t_references.result is 'full json string formatted result of crossref.org query';"]:
    conn.execute(text(stmt).execution_options(autocommit=True))

In [34]:
dfSubsites.subsite_lon = pd.to_numeric(dfSubsites.subsite_lon.str.replace("°",""))
dfSubsites.subsite_lat = pd.to_numeric(dfSubsites.subsite_lat.str.replace("°",""))
dfSubsites.drilling_angle = pd.to_numeric(dfSubsites.drilling_angle.str.replace("°",""))
dfSubsites.subsite_elevation = pd.to_numeric(dfSubsites.subsite_elevation.str.replace("m",""))
##dfSubsites.subsite_id = pd.to_numeric(dfSubsites.subsite_id).astype(int)
for c in ["calm_code","responsible_countries","responsible_person","lithology",
          "subsite_type","gtn_p","drilling_method","morphology"]:
    dfSubsites[c] = dfSubsites[c].replace(np.nan, '', regex=True)
dfSubsites.head()

,calm_code,responsible_countries,responsible_person,lithology,subsite_lon,subsite_lat,subsite_elevation,subsite_type,created,modified,subsite_id,view_url,entries,references,gtn_p,drilling_angle,drilling_method,morphology
1,U31 A,"United States,",Nikolay Shiklomanov,Organic Layer thikness:NA; mineral texture -- ...,-148.682100,69.696900,114.259277,activelayer,2014-02-03 11:00:00,2016-10-24 12:00:00,a0227,/activelayers/view/227/,"[""Walker, D.A., Auerbach, N.A., Bockheim, J.G....","[""http://dx.doi.org/10.1038/28839"", ""http://dx...",,NaN,,
1,U8,"United States,",Vladimir E. Romanovsky,The organic layer thickness is 0.23 m,-148.716667,69.683333,120.003014,activelayer,2013-12-27 15:00:00,2014-02-03 14:00:00,a0015,/activelayers/view/15/,"[""Nicolsky, D. J., Romanovsky, V. E., Alexeev,...","[""http://dx.doi.org/10.1029/2007gl029525"", ""ht...",,NaN,,
1,,"United States,",Gary D. Clow,,-146.338233,69.605883,269.993164,temperatures,2014-08-20 11:00:00,2016-06-02 16:00:00,t1128,/boreholes/view/1128/,[],[],US 94,90.0,,
1,,"United States,",Doug L. Kane,,-148.750000,69.833333,76.467224,temperatures,2015-10-06 15:00:00,2015-10-06 15:00:00,t1173,/boreholes/view/1173/,[],[],US 43,90.0,,
1,,"United States,",Vladimir E. Romanovsky,,-148.720766,69.674140,122.868607,temperatures,2013-12-09 14:00:00,2014-02-27 10:00:00,t0103,/boreholes/view/103/,[],[],,90.0,,


In [35]:
gdfSubites = gpd.GeoDataFrame(dfSubsites, 
                              geometry=gpd.points_from_xy(dfSubsites.subsite_lon,
                                                          dfSubsites.subsite_lat)).set_crs(epsg=4326)
gdfSubites

,calm_code,responsible_countries,responsible_person,lithology,subsite_lon,subsite_lat,subsite_elevation,subsite_type,created,modified,subsite_id,view_url,entries,references,gtn_p,drilling_angle,drilling_method,morphology,geometry
1,U31 A,"United States,",Nikolay Shiklomanov,Organic Layer thikness:NA; mineral texture -- ...,-148.682100,69.696900,114.259277,activelayer,2014-02-03 11:00:00,2016-10-24 12:00:00,a0227,/activelayers/view/227/,"[""Walker, D.A., Auerbach, N.A., Bockheim, J.G....","[""http://dx.doi.org/10.1038/28839"", ""http://dx...",,NaN,,,POINT (-148.68210 69.69690)
1,U8,"United States,",Vladimir E. Romanovsky,The organic layer thickness is 0.23 m,-148.716667,69.683333,120.003014,activelayer,2013-12-27 15:00:00,2014-02-03 14:00:00,a0015,/activelayers/view/15/,"[""Nicolsky, D. J., Romanovsky, V. E., Alexeev,...","[""http://dx.doi.org/10.1029/2007gl029525"", ""ht...",,NaN,,,POINT (-148.71667 69.68333)
1,,"United States,",Gary D. Clow,,-146.338233,69.605883,269.993164,temperatures,2014-08-20 11:00:00,2016-06-02 16:00:00,t1128,/boreholes/view/1128/,[],[],US 94,90.0,,,POINT (-146.33823 69.60588)
1,,"United States,",Doug L. Kane,,-148.750000,69.833333,76.467224,temperatures,2015-10-06 15:00:00,2015-10-06 15:00:00,t1173,/boreholes/view/1173/,[],[],US 43,90.0,,,POINT (-148.75000 69.83333)
1,,"United States,",Vladimir E. Romanovsky,,-148.720766,69.674140,122.868607,temperatures,2013-12-09 14:00:00,2014-02-27 10:00:00,t0103,/boreholes/view/103/,[],[],,90.0,,,POINT (-148.72077 69.67414)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,,"United States,",Kenji Yoshikawa,,-163.413000,64.682000,14.000000,temperatures,2014-01-20 13:00:00,2014-06-06 09:00:00,t0771,/boreholes/view/771/,[],[],US O-94,90.0,,,POINT (-163.41300 64.68200)
1,,"Italy,",Mauro Guglielmin,,10.308250,46.392250,2570.000000,temperatures,2014-01-22 10:00:00,2014-01-22 10:00:00,t0895,/boreholes/view/895/,[],[],IT 02,90.0,,,POINT (10.30825 46.39225)
1,,"Germany,",Andreas Poschinger,,10.987108,47.421789,2922.000000,temperatures,2014-01-21 15:00:00,2016-10-22 09:00:00,t0859,/boreholes/view/859/,[],[],DE 01,90.0,,,POINT (10.98711 47.42179)
1,,"Germany,",Andreas Poschinger,,10.987142,47.421778,2922.000000,temperatures,2014-01-21 15:00:00,2016-10-22 09:00:00,t0858,/boreholes/view/858/,[],[],DE 02,90.0,,,POINT (10.98714 47.42178)


In [36]:
# subsite_id is not unique, we need view_url as the unique key
gdfSubsiteInfo = gdfSubites.merge(gdfSites,on="view_url")

In [37]:
# consistency check (1)
gdfSubsiteInfo[gdfSubsiteInfo.subsite_type_x != gdfSubsiteInfo.subsite_type_y]

,calm_code,responsible_countries,responsible_person,lithology,subsite_lon,subsite_lat,subsite_elevation,subsite_type_x,created,modified,...,have_data,observation_type,area,site_lon,site_lat,bbox_lat_max,bbox_lat_min,bbox_lon_min,bbox_lon_max,geometry_y


In [38]:
# consistency check (2)
gdfSubsiteInfo[gdfSubsiteInfo.subsite_id_x != gdfSubsiteInfo.subsite_id_y]

,calm_code,responsible_countries,responsible_person,lithology,subsite_lon,subsite_lat,subsite_elevation,subsite_type_x,created,modified,...,have_data,observation_type,area,site_lon,site_lat,bbox_lat_max,bbox_lat_min,bbox_lon_min,bbox_lon_max,geometry_y


In [39]:
# now that we checked consistency, lets get rid of duplicates
gdfSubsiteInfo.rename(columns={"subsite_type_x":"subsite_type",
                              "subsite_id_x":"subsite_id",
                              "geometry_x":"geometry",
                              "geometry_y":"geometry_site",
                              "name":"subsite_name",
                              "site":"site_name"},
                     inplace=True)
del gdfSubsiteInfo["subsite_type_y"]
del gdfSubsiteInfo["subsite_id_y"]
gdfSubsiteInfo = gpd.GeoDataFrame(gdfSubsiteInfo).set_geometry('geometry').set_crs(epsg=4326)
gdfSubsiteInfo

,calm_code,responsible_countries,responsible_person,lithology,subsite_lon,subsite_lat,subsite_elevation,subsite_type,created,modified,...,have_data,observation_type,area,site_lon,site_lat,bbox_lat_max,bbox_lat_min,bbox_lon_min,bbox_lon_max,geometry_site
0,U31 A,"United States,",Nikolay Shiklomanov,Organic Layer thikness:NA; mineral texture -- ...,-148.682100,69.696900,114.259277,activelayer,2014-02-03 11:00:00,2016-10-24 12:00:00,...,False,"CALM, TSP",1934.328,-147.544117,69.719608,69.833333,69.605883,-148.750000,-146.338233,POINT (-147.54412 69.71961)
1,U8,"United States,",Vladimir E. Romanovsky,The organic layer thickness is 0.23 m,-148.716667,69.683333,120.003014,activelayer,2013-12-27 15:00:00,2014-02-03 14:00:00,...,True,"CALM, TSP",1934.328,-147.544117,69.719608,69.833333,69.605883,-148.750000,-146.338233,POINT (-147.54412 69.71961)
2,,"United States,",Gary D. Clow,,-146.338233,69.605883,269.993164,temperatures,2014-08-20 11:00:00,2016-06-02 16:00:00,...,False,"CALM, TSP",1934.328,-147.544117,69.719608,69.833333,69.605883,-148.750000,-146.338233,POINT (-147.54412 69.71961)
3,,"United States,",Doug L. Kane,,-148.750000,69.833333,76.467224,temperatures,2015-10-06 15:00:00,2015-10-06 15:00:00,...,False,"CALM, TSP",1934.328,-147.544117,69.719608,69.833333,69.605883,-148.750000,-146.338233,POINT (-147.54412 69.71961)
4,,"United States,",Vladimir E. Romanovsky,,-148.720766,69.674140,122.868607,temperatures,2013-12-09 14:00:00,2014-02-27 10:00:00,...,True,"CALM, TSP",1934.328,-147.544117,69.719608,69.833333,69.605883,-148.750000,-146.338233,POINT (-147.54412 69.71961)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628,,"United States,",Kenji Yoshikawa,,-163.413000,64.682000,14.000000,temperatures,2014-01-20 13:00:00,2014-06-06 09:00:00,...,False,TSP,0.000,-163.413000,64.682000,64.682000,64.682000,-163.413000,-163.413000,POINT (-163.41300 64.68200)
1629,,"Italy,",Mauro Guglielmin,,10.308250,46.392250,2570.000000,temperatures,2014-01-22 10:00:00,2014-01-22 10:00:00,...,False,TSP,0.000,10.308250,46.392250,46.392250,46.392250,10.308250,10.308250,POINT (10.30825 46.39225)
1630,,"Germany,",Andreas Poschinger,,10.987108,47.421789,2922.000000,temperatures,2014-01-21 15:00:00,2016-10-22 09:00:00,...,False,TSP,0.000,10.983662,47.420216,47.421789,47.418644,10.980181,10.987142,POINT (10.98366 47.42022)
1631,,"Germany,",Andreas Poschinger,,10.987142,47.421778,2922.000000,temperatures,2014-01-21 15:00:00,2016-10-22 09:00:00,...,False,TSP,0.000,10.983662,47.420216,47.421789,47.418644,10.980181,10.987142,POINT (10.98366 47.42022)


In [40]:
del gdfSubsiteInfo["geometry_site"]

In [41]:
stmt = """CREATE TABLE "public"."t_permafrost_subsites" (
	"calm_code" varchar(20), 
	"responsible_countries" varchar(60), 
	"responsible_person" varchar(100), 
	"lithology" varchar(1000), 
	"subsite_lon" double precision, 
	"subsite_lat" double precision, 
	"subsite_elevation" double precision, 
	"subsite_type" varchar(20), 
	"created" timestamp(6), 
	"modified" timestamp(6), 
	"subsite_id" varchar(5), 
	"view_url" varchar(100), 
	"entries" varchar(5000), 
	"references" varchar(1500), 
	"gtn_p" varchar(20), 
	"drilling_angle" double precision, 
	"drilling_method" varchar(200), 
	"morphology" varchar(1000), 
	"geometry" geometry, 
	"subsite_name" varchar(100), 
	"site_name" varchar(100), 
	"site_id" integer, 
	"site_url" varchar(100), 
	"country" varchar(40), 
	"type" varchar(20), 
	"code" varchar(20), 
	"vegetation" varchar(50), 
	"permafrost" varchar(50), 
	"elevation" double precision, 
	"iso2" varchar(2), 
	"iso3" varchar(3), 
	"country_official_name" varchar(80), 
	"depth" double precision, 
	"have_data" boolean, 
	"observation_type" varchar(20), 
	"area" double precision, 
	"site_lon" double precision, 
	"site_lat" double precision, 
	"bbox_lat_max" double precision, 
	"bbox_lat_min" double precision, 
	"bbox_lon_min" double precision, 
	"bbox_lon_max" double precision
);"""
conn.execute(text(stmt).execution_options(autocommit=True))

stmt = """CREATE INDEX "idx_t_permafrost_subsites_geometry"
	ON "public"."t_permafrost_subsites"
	USING gist (geometry);
"""
conn.execute(text(stmt).execution_options(autocommit=True))

In [42]:
gdfSubsiteInfo.to_postgis("t_permafrost_subsites",con=conn,
                          dtype={"calm_code":sqlalchemy.types.VARCHAR(20),
                                 "responsible_countries":sqlalchemy.types.VARCHAR(60),
                                 "responsible_person":sqlalchemy.types.VARCHAR(100),
                                 "lithology":sqlalchemy.types.VARCHAR(1000),
                                 "subsite_type":sqlalchemy.types.VARCHAR(20),
                                 "created":sqlalchemy.types.TIMESTAMP,
                                 "modified":sqlalchemy.types.TIMESTAMP,
                                 "subsite_id":sqlalchemy.types.VARCHAR(5),
                                 "view_url":sqlalchemy.types.VARCHAR(100),
                                 "entries":sqlalchemy.types.VARCHAR(5000),
                                 "references":sqlalchemy.types.VARCHAR(1500),
                                 "gtn_p":sqlalchemy.types.VARCHAR(20),
                                 "drilling_method":sqlalchemy.types.VARCHAR(200),
                                 "morphology":sqlalchemy.types.VARCHAR(1000),
                                 "subsite_name":sqlalchemy.types.VARCHAR(100),
                                 "site_name":sqlalchemy.types.VARCHAR(100),
                                 "site_id":sqlalchemy.types.INT,
                                 "site_url":sqlalchemy.types.VARCHAR(100),
                                 "country":sqlalchemy.types.VARCHAR(40),
                                 "type":sqlalchemy.types.VARCHAR(20),
                                 "code":sqlalchemy.types.VARCHAR(20),
                                 "vegetation":sqlalchemy.types.VARCHAR(50),
                                 "permafrost":sqlalchemy.types.VARCHAR(50),
                                 "iso2":sqlalchemy.types.VARCHAR(2),
                                 "iso3":sqlalchemy.types.VARCHAR(3),
                                 "country_official_name":sqlalchemy.types.VARCHAR(80),
                                 "observation_type":sqlalchemy.types.VARCHAR(20),
                                 },if_exists="replace")

In [43]:
comments = {'calm_code': 'a TBD code identifying some [sub]sites',
            'responsible_countries': 'country name which operates site data acquisition',
            'responsible_person':'point of contact name for dataset',
            'lithology':'properties of rock or soil at subsite',
            'subsite_lon': 'nominal longitude of [sub]site',
            'subsite_lat': 'nominal latitude of [sub]site',
            'subsite_elevation': 'elevation of subsite in meters',
            'subsite_type':'one of [activelayer|temperatures]',
            'created':'record creation datetime',
            'modified':'last record update datetime',
            'subsite_id': 'ID of subsite as generated by gtnpdatabase.org',
            'view_url': 'path portion of subsite webpage',
            'entries':'json string of array with free form text of references pertaining to the subsite or its data',
            'references':'json string of array with doi URLs for the above references',
            'gtn_p':'subsite code for temperature subsites',
            'drilling_angle':'angle of temperature borehole (90° is vertical)',
            'drilling_method':'free form text describing how hole is drille',
            'morphology':'free form text describing the landscape surrounding the site',
            'geometry': 'GIS geometry of subsite longitude/latitude',
            'subsite_name': 'Name of the subsite (usually the hole itself)',
            'site_name': 'Name of the site (usually a collection of boreholes or a place name)',
            'site_id': 'numerical ID of site as generated by gtnpdatabase.org',
            'site_url': 'path portion of site webpage',
            'country': 'country name where site is located in',
            'type': 'enumeration describing borehole layouts',
            'code': 'a TBD code identifying some (sub)sites',
            'vegetation': 'enumerated class of vegetation surrounding subsite',
            'permafrost': 'enumerated class of permafrost surrounding subsite',
            'elevation':'subsite elevation in [m]',
            'iso2': 'ISO 3166 ALPHA-2 country code where site is located in',
            'iso3': 'ISO 3166 ALPHA-3 country code where site is located in',
            'country_official_name': 'An official name of the country',
            'depth': 'depth of borehole in [m]',
            'have_data': 'flag indicating if time series data are available',
            'observation_type': 'One of TSP,CALM,or both indicating TBD',
            'area': 'area of site in [km2]',
            'site_lon': 'nominal longitude of site',
            'site_lat': 'nominal latitude of site',
            'bbox_lat_max': 'max latitude of site extension',
            'bbox_lat_min': 'min latitude of site extension',
            'bbox_lon_min': 'max longitude of site extension',
            'bbox_lon_max': 'min longitude of site extension'}

In [44]:
# https://stackoverflow.com/questions/60210704/how-can-i-insert-column-comments-in-postgresql-via-python
# 
table = "t_permafrost_subsites"
for column,comment in comments.items():
    stmt = f"COMMENT ON COLUMN {table}.{column} is '{comment}';"
    conn.execute(text(stmt).execution_options(autocommit=True))

In [45]:
#gdfSubsiteInfo = gpd.read_postgis("SELECT * FROM t_permafrost_subsites",con=conn,geom_col="geometry")

## Download Active Layer (Thawing) Data

As the subsite ID's are not unique between activelayer and borehole temperature sites, we need two loops. We don't bother to store the zip files but download data into a buffer and extract the content from there.

In [46]:
Path("./sun/data").mkdir(exist_ok=True,parents=True)

ok = []
ko = []
already_had = 0

count = 0
for subsite_id in tqdm(gdfSubsiteInfo[(gdfSubsiteInfo.have_data)&(gdfSubsiteInfo.subsite_type=="activelayer")].subsite_id.values):
    subsite_numerical_id = int(subsite_id[1:])# remove leading "a"
    count += 1
    url = "http://gtnpdatabase.org/rest/activelayers/dlpackage/{}/true".format(subsite_numerical_id) 
    Path("./sun/data/{}".format(subsite_id)).mkdir(exist_ok=True,parents=True)
    if len(os.listdir("./sun/data/{}".format(subsite_id))) > 0:
        already_had += 1
        continue
    r = requests.get(url)
    if r.ok:
        zf = zipfile.ZipFile(io.BytesIO(r.content))
        zf.extractall("./sun/data/{}".format(subsite_id))
        ok.append(subsite_id)
    else:
        ko.append(subsite_id)
    time.sleep(1)
print("Looked for {} records (already had {}), received {}. {} errors ({})".format(count-already_had,already_had,
                                                                  len(ok),len(ko),ko))

100%|██████████| 104/104 [00:00<00:00, 6410.86it/s]

Looked for 0 records (already had 104), received 0. 0 errors ([])


### Data Cleansing

The zip files contain a variety of records, many of which we don't need as we already acquired the metadata. Any observations are logged. Note that we now start ground-up as there may be more files in the zip file than subsites.

In [47]:
# taken from https://naysan.ca/2020/05/09/pandas-to-postgresql-using-psycopg2-bulk-insert-performance-benchmark/
# Many thanks, what a cool idea!
def copy_from_stringio(conn, df, table):
    """
    Here we are going save the dataframe in memory 
    and use copy_from() to copy it to the table
    """
    buffer = io.StringIO()
    df.to_csv(buffer, index=False, header=False)
    buffer.seek(0)
    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",", columns=df.columns)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    #print("copy_from_stringio() done")
    cursor.close()

In [48]:
def clean_save_activelayer(df,dataset_id,conn):
    dfOffsets = df[["offset_x","offset_y"]].copy()
    dfOffsets["idx"] = dfOffsets.index

    del df["offset_x"]
    del df["offset_y"]

    df = df.transpose()
    df.index = pd.to_datetime(df.index)
    df.index.name = None

    ddf = df.stack().reset_index().rename(columns={"level_0":"datetime_date","level_1":"idx",0:"depth_m"}).merge(dfOffsets,on="idx")
    ddf.index = ddf.datetime_date
    ddf.index.name = None
    del ddf["idx"]
    ddf["dataset_id"] = dataset_id
    ddf["subsite_type"] = "activelayer"
    
    ddf.dropna(subset=["depth_m"],inplace=True)
    ddf = ddf[ddf.depth_m >= 0.]
    ddf.depth_m = ddf.depth_m/100. # convert cm to m
    
    try:
        conn.execute("DELETE FROM t_permafrost_depth_data WHERE dataset_id={} AND subsite_type='activelayer'".format(dataset_id))
    except:
        pass
    copy_from_stringio(conn.raw_connection(),ddf,"t_permafrost_depth_data")

    return ddf

In [49]:
conn.execute("""CREATE TABLE IF NOT EXISTS t_permafrost_depth_data (
	"datetime_date" timestamp(6), 
	"depth_m" real, 
	"offset_x" real, 
	"offset_y" real, 
	"dataset_id" varchar(5),
	"subsite_type" varchar(20)
);""")

# https://stackoverflow.com/questions/60210704/how-can-i-insert-column-comments-in-postgresql-via-python
# 
for stmt in ["COMMENT ON COLUMN t_permafrost_depth_data.datetime_date is 'Timestamp of observation';",
             "COMMENT ON COLUMN t_permafrost_depth_data.depth_m is 'Depth in [m] where ground was frozen';",
             "COMMENT ON COLUMN t_permafrost_depth_data.offset_x is 'x offset in [m] of borehole grid';",
             "COMMENT ON COLUMN t_permafrost_depth_data.offset_y is 'y offset in [m] of borehole grid';",
             "COMMENT ON COLUMN t_permafrost_depth_data.dataset_id is 'Dataset ID, unique for activelayer data only';",
             "COMMENT ON COLUMN t_permafrost_depth_data.subsite_type is 'set to activelayer';"]:
    conn.execute(text(stmt).execution_options(autocommit=True))

In [50]:
catalogue = []

for p in tqdm(glob.glob("./sun/data/a*/*.csv")):
    folder,f = os.path.split(p)
    subsite_id = folder.split("/")[-1]
    
    data = {}
    from_dt = None
    to_dt = None
    num_records = -1
    num_years = 0
    avg_measurements_year = 0
    dataset_id = f[f.find("Dataset_")+len("Dataset_"):].split("-")[0]
    
    if not "moisture" in f:
        try:
            dfData = pd.read_csv(p).replace(-999.0,np.nan)
        except Exception as e:
            log_finding({"topic":"active layer depths",
                         "subsite_id":subsite_id,
                         "dataset_id":dataset_id,
                         "filename":f,
                         "diagnosis":"{}".format(sys.exc_info()).strip(),
                         "fix":"skipping file",
                         "needs_attention":True})
            continue
            
        if isinstance(dfData.index, pd.MultiIndex): # This was a weird error in one file which did not raise an exception
            log_finding({"topic":"active layer depths",
                         "subsite_id":subsite_id,
                         "dataset_id":dataset_id,
                         "filename":f,
                         "diagnosis":"something is not right in this file, partially more data column(s) than columns result in a MultiIndex",
                         "fix":"skipping file",
                         "needs_attention":True})
            continue

        dfData = clean_save_activelayer(dfData,dataset_id,conn)
        from_dt = dfData.datetime_date.min()
        to_dt = dfData.datetime_date.max()
        num_records = len(dfData)
        dfByYear = dfData.groupby(pd.Grouper(freq="Y")).size().replace(0,np.nan).dropna()
        num_years = len(dfByYear)
        avg_measurements_year = dfByYear.quantile(0.5)
            
        #dataset = f.replace("Ayach-Yakha","Ayach~Yakha").split("-")[2].replace("Dataset_","")
        parameter = "depth_m"
    elif "moisture" in f:
        log_finding({"topic":"active layer depths",
                     "dataset_id":dataset_id,
                     "subsite_id":subsite_id,
                     "site":f.split("-")[1],
                     "filename":f,
                     "diagnosis":"dataset contains moisture, not temperature, data",
                     "fix":"ignored",
                     "needs_attention":False})
        continue
    else:
        print("Unknown File {}".format(os.path.join(r,f)))
        continue
    catalogue.append({"filename":f,"extension":".csv","subsite_id":subsite_id,"dataset_id":dataset_id,"from_dt":from_dt,
                      "to_dt":to_dt,"num_records":num_records,"parameter":parameter,"path":folder,
                      "site":f.split("-")[1],"num_years":num_years,"avg_measurements_year":avg_measurements_year,
                      "subsite_type":"activelayer"})
                              
dfCatalogue = pd.DataFrame(catalogue)
#dfCatalogue.subsite_id = pd.to_numeric(dfCatalogue.subsite_id,errors="coerce")
#dfCatalogue.dropna(axis=0,subset=["subsite_id"],inplace=True)
dfCatalogue

100%|██████████| 106/106 [00:05<00:00, 20.21it/s]


,filename,extension,subsite_id,dataset_id,from_dt,to_dt,num_records,parameter,path,site,num_years,avg_measurements_year,subsite_type
0,Activelayer_36-Plosky_Tolbachik_1-Dataset_244-...,.csv,a0036,244,2003-06-09,2012-09-19,1325,depth_m,./sun/data/a0036,Plosky_Tolbachik_1,10,121.0,activelayer
1,Activelayer_22-Council-Dataset_286-Sporadic-Un...,.csv,a0022,286,1999-10-03,2019-08-25,1646,depth_m,./sun/data/a0022,Council,16,100.5,activelayer
2,Activelayer_101-Bykovsky_Cape_Plakor-Dataset_2...,.csv,a0101,219,2001-07-28,2014-09-09,1430,depth_m,./sun/data/a0101,Bykovsky_Cape_Plakor,12,121.0,activelayer
3,Activelayer_250-Urengoy_GAS_FIELD_GP5-Dataset_...,.csv,a0250,1524,2008-09-04,2019-08-20,1571,depth_m,./sun/data/a0250,Urengoy_GAS_FIELD_GP5,12,121.0,activelayer
4,Activelayer_96-Kuropatochya_River_R12b-Dataset...,.csv,a0096,248,1996-08-26,1996-08-26,121,depth_m,./sun/data/a0096,Kuropatochya_River_R12b,1,121.0,activelayer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Activelayer_83-Lake_Akhmelo-Dataset_261-Sporad...,.csv,a0083,261,1996-09-12,2018-08-31,2649,depth_m,./sun/data/a0083,Lake_Akhmelo,21,121.0,activelayer
88,Activelayer_107-Yubileynoe_3_WET-Dataset_273-S...,.csv,a0107,273,2007-07-09,2007-07-09,121,depth_m,./sun/data/a0107,Yubileynoe_3_WET,1,121.0,activelayer
89,Activelayer_86-Mt__Rodinka_GRID-Dataset_256-Sp...,.csv,a0086,256,1998-09-27,2019-09-21,2662,depth_m,./sun/data/a0086,Mt__Rodinka_GRID,22,121.0,activelayer
90,Activelayer_112-Vaskiny_Dachi_2-Dataset_1521-S...,.csv,a0112,1521,2007-09-01,2019-08-26,712,depth_m,./sun/data/a0112,Vaskiny_Dachi_2,13,55.0,activelayer


In [51]:
dfCatalogue.to_parquet("./sun/activelayers/datasets.parquet")

## Download Active Layer Temperature Data

This dataset often contains more than one dataset per folder as there also exist e.g. satellite surface temperatures. These can be acquired via https://modis.gsfc.nasa.gov/data/, which may undergo data product updates and is suggested as the ground truth reference dataset.

In [52]:
Path("./sun/data").mkdir(exist_ok=True,parents=True)

ok = []
ko = []
already_had = 0

count = 0
for subsite_id in tqdm(gdfSubsiteInfo[(gdfSubsiteInfo.have_data)&(gdfSubsiteInfo.subsite_type=="temperatures")].subsite_id.values):
    subsite_numerical_id = int(subsite_id[1:])# remove leading "a"
    count += 1
    url = "http://gtnpdatabase.org/rest/boreholes/dlpackage/{}/true".format(subsite_numerical_id)
    Path("./sun/data/{}".format(subsite_id)).mkdir(exist_ok=True,parents=True)
    if len(os.listdir("./sun/data/{}".format(subsite_id))) > 0:
        already_had += 1
        continue
    r = requests.get(url)
    if r.ok:
        zf = zipfile.ZipFile(io.BytesIO(r.content))
        zf.extractall("./sun/data/{}".format(subsite_id))
        ok.append(subsite_id)
    else:
        ko.append(subsite_id)
    time.sleep(0.25)
print("Looked for {} records (already had {}), received {}. {} errors ({})".format(count-already_had,already_had,
                                                                  len(ok),len(ko),ko))

100%|██████████| 498/498 [00:00<00:00, 7245.53it/s]

Looked for 0 records (already had 498), received 0. 0 errors ([])


In [53]:
alldata = []

#for r,dd,ff in os.walk("./sun/boreholes/data"):
#    if ".ipynb_checkpoints" in r:
#        continue
#    for f in ff:
for p in tqdm(glob.glob("./sun/data/t*/*.csv")):
    folder,f = os.path.split(p)
    subsite_id = folder.split("/")[-1]

    part_1,part_2 = re.split("-Dataset_",f)

    fields = part_2.split("-")
    entry = dict(zip(["field_{}".format(i) for i in range(len(fields))],fields))
    entry["subsite_id"]=subsite_id #int(part_1.split("-")[0].replace("Borehole_",""))
    entry["filename"] = f
    entry["path"] = p
    entry["extension"] = ".csv"

    if entry["field_3"] == "Ground_Temperature":
        alldata.append(entry)
    else:
        entry = {}
                
dfCatalogueTemps = pd.DataFrame(alldata).rename(columns={"field_0":"dataset_id",
                                                  "field_1":"aggregation",
                                                  "field_2":"aggregation_frequency",
                                                  "field_3":"parameter",
                                                  "field_4":"instrument"})
dfCatalogueTemps.instrument = dfCatalogueTemps.instrument.str.replace(".timeserie.csv","")
dfCatalogueTemps

100%|██████████| 1542/1542 [00:00<00:00, 242960.81it/s]


,dataset_id,aggregation,aggregation_frequency,parameter,instrument,subsite_id,filename,path,extension
0,2006,Average,Daily,Ground_Temperature,Thermistor_Automated,t0873,Borehole_873-GEM_0106-Dataset_2006-Average-Dai...,./sun/data/t0873/Borehole_873-GEM_0106-Dataset...,.csv
1,1940,Average,Monthly,Ground_Temperature,Unknown,t1697,Borehole_1697-Makushino-Dataset_1940-Average-M...,./sun/data/t1697/Borehole_1697-Makushino-Datas...,.csv
2,1497,Continuous,Hourly,Ground_Temperature,Thermistor_Automated,t0062,Borehole_62-Breinosa__E-2009_-Dataset_1497-Con...,./sun/data/t0062/Borehole_62-Breinosa__E-2009_...,.csv
3,29,Continuous,6_hours,Ground_Temperature,Thermistor_Automated,t0057,Borehole_57-Kapp_Linne_1-Dataset_29-Continuous...,./sun/data/t0057/Borehole_57-Kapp_Linne_1-Data...,.csv
4,2047,Constant_Over_Interval,Hourly,Ground_Temperature,Thermistor_Automated,t0449,Borehole_449-Udachny_1-Dataset_2047-Constant_O...,./sun/data/t0449/Borehole_449-Udachny_1-Datase...,.csv
...,...,...,...,...,...,...,...,...,...
429,1850,Average,Monthly,Ground_Temperature,Unknown,t1607,Borehole_1607-Khanty-Mansiisk-Dataset_1850-Ave...,./sun/data/t1607/Borehole_1607-Khanty-Mansiisk...,.csv
430,665,Constant_Over_Interval,Daily,Ground_Temperature,Unknown,t0105,Borehole_105-Franklin_Bluffs__dry__ib-Dataset_...,./sun/data/t0105/Borehole_105-Franklin_Bluffs_...,.csv
431,186,Constant_Over_Interval,Daily,Ground_Temperature,Unknown,t0189,Borehole_189-Vorkuta_RU0040-Dataset_186-Consta...,./sun/data/t0189/Borehole_189-Vorkuta_RU0040-D...,.csv
432,1502,Average,Daily,Ground_Temperature,Thermistor_Automated,t0464,Borehole_464-Kamchatka_1_06-Dataset_1502-Avera...,./sun/data/t0464/Borehole_464-Kamchatka_1_06-D...,.csv


In [54]:
dfCatalogueTemps.to_parquet("./sun/boreholes/datasets.parquet")

### Data Cleansing

In [55]:
dfCatalogueTemps = pd.read_parquet("./sun/boreholes/datasets.parquet")

In [56]:
def compute_actual_data_frequency(dfData):
    datetime_date = pd.Series(dfData.index)
    offset = abs((datetime_date-datetime_date.shift(-1)).dt.total_seconds().median())/86400.
    if (350. <= offset) and (offset <= 385.):
        aggregation_frequency = "Annually"
        expected_frequency = 365.25*24*3600
        resample_alias = "Y"
    elif (90. <= offset) and (offset <= 92.):
        aggregation_frequency = "Quarterly"
        resample_alias = "3M"
        expected_frequency = np.mean([31+28.25+31,30+31+30,31+31+30,31+30+31])*24*3600
    elif (30. <= offset) and (offset <= 31.1):
        aggregation_frequency = "Monthly"
        expected_frequency = np.mean([31,28.25,31,30,31,30,31,31,30,31,30,31])*24*3600
        resample_alias = "M"
    elif (.99 <= offset) and (offset <= 1.01):
        aggregation_frequency = "Daily"
        expected_frequency = 24*3600
        resample_alias = "D"
    elif (.49 <= offset) and (offset <= .51):
        aggregation_frequency = "12_hours"
        expected_frequency = 12*3600
        resample_alias = "12H"
    elif (.24 <= offset) and (offset <= .26):
        aggregation_frequency = "6_hours"
        expected_frequency = 6*3600
        resample_alias = "6H"
    elif (.16 <= offset) and (offset <= .17):
        aggregation_frequency = "4_hours"
        expected_frequency = 2*3600
        resample_alias = "4H"
    elif (.12 <= offset) and (offset <= .13):
        aggregation_frequency = "3_hours"
        expected_frequency = 3*3600
        resample_alias = "3H"
    elif (.08 <= offset) and (offset <= .085):
        aggregation_frequency = "2_hours"
        expected_frequency = 2*3600
        resample_alias = "2H"
    elif (.04 <= offset) and (offset <= .045):
        aggregation_frequency = "Hourly"
        expected_frequency = 1*3600
        resample_alias = "H"
    else:
        aggregation_frequency = "TBD"
        print("cannot compute frequency",end=" ")
        expected_frequency = 0.
        resample_alias = ""

    return {"aggregation_frequency":aggregation_frequency,"expected_frequency":expected_frequency,"resample_alias":resample_alias,"offset":offset}

In [57]:
def compute_gaps_and_dropouts(df,frequency_info):
    xdf = df.copy()
    xdf["datetime_date"] = xdf.index
    gaps = pd.DataFrame(xdf.datetime_date.diff(1).dt.total_seconds()/frequency_info['expected_frequency'])
    gaps.columns = ["missing_periods"]
    gaps = gaps.round(0).dropna()
    gaps["missing_periods"] = gaps["missing_periods"].astype(int)-1
    gaps = gaps[gaps["missing_periods"] > 0]
    dropouts = gaps[gaps["missing_periods"]==1].rename(columns={"missing_periods":"dropouts"})
    gaps = gaps[gaps["missing_periods"] > 1]
    return dropouts,gaps

In [58]:
def clean_save_temperatures(df,dataset_id,subsite_id,filename,conn):
    for c in df.columns:
        if "Date" in c:
            continue
        else:
            value_columns.append(c)
    df[value_columns] = df[value_columns].apply(lambda x: np.where(x < -273.15,np.nan,x))
    
    df.index = pd.to_datetime(df["Date/Depth"])
    df.index.name = None
    del df["Date/Depth"]
    df = df.apply(lambda x: np.where(x < -273.15,np.nan,x)).dropna(axis=0,how="all")
    
    frequencies = compute_actual_data_frequency(df)
    if frequencies["aggregation_frequency"] == "TBD":
        log_finding({"topic":"borehole temperatures",
                     "dataset_id":dataset_id,
                     "subsite_id":subsite_id,
                     "filename":filename,
                     "diagnosis":"dataset with irregular time resolution which cannot be resolved.",
                     "fix":"Skip dataset and ignoring {} records.".format(len(df)),
                     "needs_attention":True
                    })
        return pd.DataFrame(),{}
    coverage = (df.index.max()-df.index.min()).total_seconds()
    expected_num_records = coverage/frequencies["expected_frequency"]+1
    actual_num_records = len(df)
    completeness = (actual_num_records)/expected_num_records
    
    dropouts = pd.DataFrame({"dropouts":[]})
    gaps = pd.DataFrame({"missing_periods":[]})
    if completeness > 1.001:
        df = df.resample(frequencies["resample_alias"]).mean()
        log_finding({"topic":"borehole temperatures",
                     "filename":filename,
                     "dataset_id":dataset_id,
                     "subsite_id":subsite_id,
                     "diagnosis":"duplicate datetimes",
                     "fix":"resampled dataframe to aggregation frequency {}(pandas: {}). Old number of records {}, new number of records {}.".format(frequencies['aggregation_frequency'],
                                                                                                                                                     frequencies['resample_alias'],
                                                                                                                                                     actual_num_records,
                                                                                                                                                     len(df)),
                     "needs_attention":True
                    })
        new_completeness = (len(df))/expected_num_records
        #print(dataset_id,actual_num_records,completeness,frequencies,len(df),new_completeness)
        #assert False
    elif completeness < 0.99:
        dropouts,gaps = compute_gaps_and_dropouts(df,frequencies)
        if len(dropouts)>0:
            log_finding({"topic":"borehole temperatures",
                     "filename":filename,
                         "dataset_id":dataset_id,
                         "subsite_id":subsite_id,
                         "diagnosis":"missing data: saw {} dropouts in {} records".format(len(dropouts),len(df)),
                         "fix":"not possible",
                         "needs_attention":True
                        })
        if len(gaps)>0:
            log_finding({"topic":"borehole temperatures",
                     "filename":filename,
                         "dataset_id":dataset_id,
                         "subsite_id":subsite_id,
                         "diagnosis":"missing data: saw {} gaps".format(len(gaps)),
                         "fix":"not possible",
                         "needs_attention":True
                        })
    
    deficiencies = {"gaps_ratio":gaps.sum().missing_periods/expected_num_records,
                    "longest_gap":gaps.max().missing_periods,
                    "avg_gap_d":gaps.mean().missing_periods*frequencies['expected_frequency']/86400,
                    "avg_gap_h":gaps.mean().missing_periods*frequencies['expected_frequency']/24,
                    "dropouts_ratio":dropouts.sum().dropouts/(len(df)+gaps.sum().missing_periods)}
    
    #df = df.apply(lambda x: np.where(x < -273.15,np.nan,x)).dropna(axis=0,how="all")
    ddf = df.stack().reset_index().rename(columns={"level_0":"datetime_date","level_1":"depth_m",0:"temperature_degC"})
    ddf.depth_m = pd.to_numeric(ddf.depth_m)
    ddf.temperature_degC = pd.to_numeric(ddf.temperature_degC)
    ddf.dropna(axis=0,subset=["temperature_degC"],inplace=True)
    
    ddf["dataset_id"] = dataset_id
    
    ddf.index = ddf.datetime_date
    ddf.index.name = None    
    
    #return ddf

    try:
        conn.execute("DELETE FROM t_permafrost_temperature_data WHERE dataset_id={}".format(dataset_id))
    except:
        pass
    
    # too slow
    #ddf.to_sql("t_permafrost_temperature_data",con=conn,if_exists="append",dtype={"datetime_date":sqlalchemy.types.TIMESTAMP,
    #                                                                              "depth_m":sqlalchemy.types.REAL,
    #                                                                              "temperature_degC":sqlalchemy.types.REAL,
    #                                                                              "dataset_id":sqlalchemy.types.INTEGER},
    #          index=False)
    # use this
    copy_from_stringio(conn.raw_connection(),ddf,"t_permafrost_temperature_data")
    
    return ddf, deficiencies

In [59]:
stmt = """CREATE TABLE IF NOT EXISTS t_permafrost_temperature_data (
	"datetime_date" timestamp(6), 
	"depth_m" real, 
	"temperature_degc" real, 
	"dataset_id" varchar(5)
);"""

conn.execute(text(stmt).execution_options(autocommit=True))

In [60]:
alldata = []
for i,r in tqdm(dfCatalogueTemps.iterrows()):
    
    path = r.path
    ddf = pd.read_csv(path)
    value_columns = []
    if len(ddf) <= 0:
        log_finding({"topic":"borehole temperatures",
                     "dataset":r.dataset_id,
                    "subsite_id":r.subsite_id,
                    "filename":r.filename,
                    "diagnosis":"file contains no data",
                    "fix":"ignored",
                    "needs_attention":True})
        continue
        
    dfData,deficiencies = clean_save_temperatures(ddf,r.dataset_id,r.subsite_id,r.filename,conn)
    
    if len(dfData.columns) <= 0:
        log_finding({"topic":"borehole temperatures",
                     "dataset":r.dataset_id,
                    "subsite_id":r.subsite_id,
                    "filename":r.filename,
                    "diagnosis":"file contains unknown time resolution or too many gaps",
                    "fix":"ignored, see previous message",
                    "needs_attention":True})
        continue
    elif len(dfData) <= 0:
        log_finding({"topic":"borehole temperatures",
                     "dataset":r.dataset_id,
                    "subsite_id":r.subsite_id,
                    "filename":r.filename,
                    "diagnosis":"file contains no valid data, only entries less than -273.15 deg C",
                    "fix":"ignored",
                    "needs_attention":True})
        continue
    
    from_dt = dfData.datetime_date.min()
    to_dt = dfData.datetime_date.max()
    num_records = len(dfData)
    dfByYear = dfData.groupby(pd.Grouper(freq="Y")).size().replace(0,np.nan).dropna()
    num_years = len(dfByYear)
    avg_measurements_year = dfByYear.quantile(0.5)

    parameter = "temperature_degC"

    new_entry = {"filename":r.filename,"extension":r.extension,"subsite_id":r.subsite_id,"dataset_id":r.dataset_id,"from_dt":from_dt,
                          "to_dt":to_dt,"num_records":num_records,"parameter":parameter,"path":r.path,
                          "site":r.filename.split("-")[1],"num_years":num_years,"avg_measurements_year":avg_measurements_year,
                          "subsite_type":"temperatures"}
    new_entry.update(deficiencies)
    alldata.append(new_entry)
#column_names
pd.DataFrame(alldata)

72it [00:14,  6.57it/s]

cannot compute frequency 

80it [00:18,  3.18it/s]

cannot compute frequency 

85it [00:19,  4.55it/s]

...

cannot compute frequency 

426it [01:19,  5.44it/s]

cannot compute frequency 

434it [01:20,  5.39it/s]


,filename,extension,subsite_id,dataset_id,from_dt,to_dt,num_records,parameter,path,site,num_years,avg_measurements_year,subsite_type,gaps_ratio,longest_gap,avg_gap_d,avg_gap_h,dropouts_ratio
0,Borehole_873-GEM_0106-Dataset_2006-Average-Dai...,.csv,t0873,2006,2006-10-05 00:00:00,2018-09-12 00:00:00,96027,temperature_degC,./sun/data/t0873/Borehole_873-GEM_0106-Dataset...,GEM_0106,13,6885.0,temperatures,0.035542,99.0,38.750000,139500.0,0.000000
1,Borehole_1697-Makushino-Dataset_1940-Average-M...,.csv,t1697,1940,1930-01-15 12:00:00,1990-12-15 12:00:00,3190,temperature_degC,./sun/data/t1697/Borehole_1697-Makushino-Datas...,Makushino,60,67.0,temperatures,0.049182,12.0,121.750000,438300.0,0.019499
2,Borehole_62-Breinosa__E-2009_-Dataset_1497-Con...,.csv,t0062,1497,2009-04-01 00:00:00,2015-09-06 10:00:00,546667,temperature_degC,./sun/data/t0062/Borehole_62-Breinosa__E-2009_...,Breinosa__E,7,87475.0,temperatures,0.001348,70.0,1.583333,5700.0,0.029350
3,Borehole_57-Kapp_Linne_1-Dataset_29-Continuous...,.csv,t0057,29,2008-09-23 12:00:00,2020-09-15 06:00:00,366759,temperature_degC,./sun/data/t0057/Borehole_57-Kapp_Linne_1-Data...,Kapp_Linne_1,13,30657.0,temperatures,0.000000,NaN,NaN,NaN,0.000000
4,Borehole_449-Udachny_1-Dataset_2047-Constant_O...,.csv,t0449,2047,2013-11-14 11:59:00,2014-08-24 20:59:00,27208,temperature_degC,./sun/data/t0449/Borehole_449-Udachny_1-Datase...,Udachny_1,2,13604.0,temperatures,0.000000,NaN,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,Borehole_1607-Khanty-Mansiisk-Dataset_1850-Ave...,.csv,t1607,1850,1961-08-15 12:00:00,1990-12-15 12:00:00,2061,temperature_degC,./sun/data/t1607/Borehole_1607-Khanty-Mansiisk...,Khanty,30,70.0,temperatures,0.005666,2.0,60.875000,219150.0,0.005698
403,Borehole_105-Franklin_Bluffs__dry__ib-Dataset_...,.csv,t0105,665,2006-06-01 00:00:00,2013-05-31 00:00:00,39376,temperature_degC,./sun/data/t0105/Borehole_105-Franklin_Bluffs_...,Franklin_Bluffs__dry__ib,8,5307.0,temperatures,0.000000,NaN,NaN,NaN,0.000000
404,Borehole_189-Vorkuta_RU0040-Dataset_186-Consta...,.csv,t0189,186,2008-08-20 00:00:00,2011-08-25 00:00:00,4404,temperature_degC,./sun/data/t0189/Borehole_189-Vorkuta_RU0040-D...,Vorkuta_RU0040,4,1204.0,temperatures,0.000000,NaN,NaN,NaN,0.000000
405,Borehole_464-Kamchatka_1_06-Dataset_1502-Avera...,.csv,t0464,1502,2007-09-17 00:00:00,2011-08-04 00:00:00,4234,temperature_degC,./sun/data/t0464/Borehole_464-Kamchatka_1_06-D...,Kamchatka_1_06,5,1084.0,temperatures,0.000000,NaN,NaN,NaN,0.000000


In [61]:
dfCatalogueTempsComplete = dfCatalogueTemps.merge(pd.DataFrame(alldata), 
                                                  on="dataset_id",
                                                  suffixes=('', '_dup'))
dfCatalogueTempsComplete.drop(dfCatalogueTempsComplete.filter(regex='_dup$').columns.tolist(),axis=1, inplace=True)
dfCatalogueTempsComplete

,dataset_id,aggregation,aggregation_frequency,parameter,instrument,subsite_id,filename,path,extension,from_dt,...,num_records,site,num_years,avg_measurements_year,subsite_type,gaps_ratio,longest_gap,avg_gap_d,avg_gap_h,dropouts_ratio
0,2006,Average,Daily,Ground_Temperature,Thermistor_Automated,t0873,Borehole_873-GEM_0106-Dataset_2006-Average-Dai...,./sun/data/t0873/Borehole_873-GEM_0106-Dataset...,.csv,2006-10-05 00:00:00,...,96027,GEM_0106,13,6885.0,temperatures,0.035542,99.0,38.750000,139500.0,0.000000
1,1940,Average,Monthly,Ground_Temperature,Unknown,t1697,Borehole_1697-Makushino-Dataset_1940-Average-M...,./sun/data/t1697/Borehole_1697-Makushino-Datas...,.csv,1930-01-15 12:00:00,...,3190,Makushino,60,67.0,temperatures,0.049182,12.0,121.750000,438300.0,0.019499
2,1497,Continuous,Hourly,Ground_Temperature,Thermistor_Automated,t0062,Borehole_62-Breinosa__E-2009_-Dataset_1497-Con...,./sun/data/t0062/Borehole_62-Breinosa__E-2009_...,.csv,2009-04-01 00:00:00,...,546667,Breinosa__E,7,87475.0,temperatures,0.001348,70.0,1.583333,5700.0,0.029350
3,29,Continuous,6_hours,Ground_Temperature,Thermistor_Automated,t0057,Borehole_57-Kapp_Linne_1-Dataset_29-Continuous...,./sun/data/t0057/Borehole_57-Kapp_Linne_1-Data...,.csv,2008-09-23 12:00:00,...,366759,Kapp_Linne_1,13,30657.0,temperatures,0.000000,NaN,NaN,NaN,0.000000
4,2047,Constant_Over_Interval,Hourly,Ground_Temperature,Thermistor_Automated,t0449,Borehole_449-Udachny_1-Dataset_2047-Constant_O...,./sun/data/t0449/Borehole_449-Udachny_1-Datase...,.csv,2013-11-14 11:59:00,...,27208,Udachny_1,2,13604.0,temperatures,0.000000,NaN,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,1850,Average,Monthly,Ground_Temperature,Unknown,t1607,Borehole_1607-Khanty-Mansiisk-Dataset_1850-Ave...,./sun/data/t1607/Borehole_1607-Khanty-Mansiisk...,.csv,1961-08-15 12:00:00,...,2061,Khanty,30,70.0,temperatures,0.005666,2.0,60.875000,219150.0,0.005698
403,665,Constant_Over_Interval,Daily,Ground_Temperature,Unknown,t0105,Borehole_105-Franklin_Bluffs__dry__ib-Dataset_...,./sun/data/t0105/Borehole_105-Franklin_Bluffs_...,.csv,2006-06-01 00:00:00,...,39376,Franklin_Bluffs__dry__ib,8,5307.0,temperatures,0.000000,NaN,NaN,NaN,0.000000
404,186,Constant_Over_Interval,Daily,Ground_Temperature,Unknown,t0189,Borehole_189-Vorkuta_RU0040-Dataset_186-Consta...,./sun/data/t0189/Borehole_189-Vorkuta_RU0040-D...,.csv,2008-08-20 00:00:00,...,4404,Vorkuta_RU0040,4,1204.0,temperatures,0.000000,NaN,NaN,NaN,0.000000
405,1502,Average,Daily,Ground_Temperature,Thermistor_Automated,t0464,Borehole_464-Kamchatka_1_06-Dataset_1502-Avera...,./sun/data/t0464/Borehole_464-Kamchatka_1_06-D...,.csv,2007-09-17 00:00:00,...,4234,Kamchatka_1_06,5,1084.0,temperatures,0.000000,NaN,NaN,NaN,0.000000


In [62]:
dfCatalogueTempsComplete.to_parquet("./sun/boreholes/datasets.complete.parquet")

## Generate Dataset Catalogue

This is allowing the combination of a 1:1 relationship between subsite and activelayer datasets, and a possible 1:many relationship between subsite and temperature measurements.

In [63]:
dfCompleteCatalogue = dfCatalogue.append(dfCatalogueTempsComplete)
for c in ["gaps_ratio","longest_gap","avg_gap_d","avg_gap_h","dropouts_ratio"]:
    dfCompleteCatalogue[c] = dfCompleteCatalogue[c].fillna(0)
#dfCompleteCatalogue.dataset_id = dfCompleteCatalogue.dataset.astype(int)
idx = dfCompleteCatalogue[dfCompleteCatalogue.subsite_type == "activelayer"].index
dfCompleteCatalogue["aggregation"].fillna("",inplace=True)
dfCompleteCatalogue["aggregation_frequency"].fillna("",inplace=True)
dfCompleteCatalogue["instrument"].fillna("",inplace=True)
dfCompleteCatalogue = dfCompleteCatalogue.rename(columns={"site":"subsite_name"})
dfCompleteCatalogue.columns

Index(['filename', 'extension', 'subsite_id', 'dataset_id', 'from_dt', 'to_dt',
       'num_records', 'parameter', 'path', 'subsite_name', 'num_years',
       'avg_measurements_year', 'subsite_type', 'aggregation',
       'aggregation_frequency', 'instrument', 'gaps_ratio', 'longest_gap',
       'avg_gap_d', 'avg_gap_h', 'dropouts_ratio'],
      dtype='object')

In [64]:
stmt = """CREATE TABLE IF NOT EXISTS t_permafrost_catalogue (
    filename VARCHAR(160),
    extension VARCHAR(4),
    subsite_id VARCHAR(5),
    dataset_id VARCHAR(5),
    from_dt TIMESTAMP(6),
    to_dt TIMESTAMP(6),
    num_records INT,
    parameter VARCHAR(18),
    path VARCHAR(200),
    subsite_name VARCHAR(60),
    num_years DOUBLE PRECISION,
    avg_measurements_year DOUBLE PRECISION,
    subsite_type VARCHAR(12),
    aggregation VARCHAR(30),
    aggregation_frequency VARCHAR(10),
    instrument VARCHAR(30),
    gaps_ratio DOUBLE PRECISION,
    longest_gap DOUBLE PRECISION,
    avg_gap_d DOUBLE PRECISION,
    avg_gap_h DOUBLE PRECISION,
    dropouts_ratio DOUBLE PRECISION
);
"""
conn.execute(text(stmt).execution_options(autocommit=True))

In [65]:
dfCompleteCatalogue.to_sql("t_permafrost_catalogue",conn,dtype={"filename":sqlalchemy.types.VARCHAR(160),
                                                                "extension":sqlalchemy.types.VARCHAR(4),
                                                                "subsite_id":sqlalchemy.types.VARCHAR(5),
                                                                "dataset_id":sqlalchemy.types.VARCHAR(5),
                                                                "from_dt":sqlalchemy.types.TIMESTAMP,
                                                                "to_dt":sqlalchemy.types.TIMESTAMP,
                                                                "num_records":sqlalchemy.types.INT,
                                                                "parameter":sqlalchemy.types.VARCHAR(18),
                                                                "path":sqlalchemy.types.VARCHAR(200),
                                                                "site":sqlalchemy.types.VARCHAR(60),
                                                                #"num_years":sqlalchemy.types.DOUBLE,
                                                                #"avg_measurements_year":sqlalchemy.types.DOUBLE,
                                                                "subsite_type":sqlalchemy.types.VARCHAR(12),
                                                                "aggregation":sqlalchemy.types.VARCHAR(30),
                                                                "aggregation_frequency":sqlalchemy.types.VARCHAR(10),
                                                                "instrument":sqlalchemy.types.VARCHAR(30),
                                                                #"gaps_ratio":sqlalchemy.types.DOUBLE,
                                                                #"longest_gap":sqlalchemy.types.DOUBLE,
                                                                #"avg_gap_d":sqlalchemy.types.DOUBLE,
                                                                #"avg_gap_h":sqlalchemy.types.DOUBLE,
                                                                #"dropouts_ratio":sqlalchemy.types.DOUBLE
                                                               },if_exists="replace")

In [66]:
Path("./mercury").mkdir(exist_ok=True,parents=True)
dfCompleteCatalogue.to_parquet("./mercury/catalogue.parquet")

## Compute Summer Active Layer Depths

In [67]:
datasets = pd.read_sql("SELECT DISTINCT dataset_id FROM t_permafrost_temperature_data;",con=conn).dataset_id.values

In [68]:
alldfs = []
for dataset_id in tqdm(datasets):
    query = """SELECT datetime_date,date_part('year'::text, t_permafrost_temperature_data.datetime_date) AS year,
    date_part('month'::text, t_permafrost_temperature_data.datetime_date) AS month,
    depth_m, temperature_degc
    FROM t_permafrost_temperature_data 
    WHERE dataset_id='{}' AND date_part('month'::text, t_permafrost_temperature_data.datetime_date) in (6,7,8,9)""".format(dataset_id)

    # WHERE dataset_id={}""".format(dataset)
    dfData = pd.read_sql(query,con=conn)

    datetime_dates = []
    zerocrossings = []

    alldata = []

    dt_slices = dfData.datetime_date.unique()
    for dt_slice in dt_slices:
        ddf = dfData[dfData.datetime_date == dt_slice]
        values = ddf.temperature_degc.values
        if values.min() > 0 or values.max() < 0:
            continue
        else:
            depths = ddf.depth_m.values
            #print(values,depths)
            f = interp1d(values,depths) #pd.to_numeric(values.index))
            zerocrossing = f(0.)[()]
            if zerocrossing < 0:
                continue
        alldata.append({"datetime_date":dt_slice,"depth_m":zerocrossing,"dataset_id":dataset_id,
                        "year":ddf.year.unique()[0],"month":ddf.month.unique()[0]})
    ddf = pd.DataFrame(alldata).dropna()
    
    alldfs.append(ddf)
    
dfActiveLayerDepthsFromTemps = pd.DataFrame().append(alldfs)
dfActiveLayerDepthsFromTemps

100%|██████████| 407/407 [04:07<00:00,  1.64it/s]


,datetime_date,depth_m,dataset_id,year,month
0,1957-06-15 00:00:00,1.150000,1944,1957.0,6.0
1,1953-08-15 12:00:00,2.361905,1944,1953.0,8.0
2,1953-09-15 00:00:00,2.311111,1944,1953.0,9.0
3,1954-06-15 00:00:00,1.533333,1944,1954.0,6.0
4,1954-07-15 12:00:00,1.866667,1944,1954.0,7.0
...,...,...,...,...,...
1038,2015-08-11 00:00:00,2.209302,1964,2015.0,8.0
1039,2015-08-12 00:00:00,2.276596,1964,2015.0,8.0
1040,2015-08-13 00:00:00,2.298969,1964,2015.0,8.0
1041,2006-08-21 00:00:00,1.136986,1964,2006.0,8.0


In [69]:
dfActiveLayerDepthsFromTemps.to_parquet("./mercury/activelayer.from.temperatures.complete.parquet")

In [70]:
#dfActiveLayerDepthsFromTempsReduced = dfActiveLayerDepthsFromTemps.groupby(["dataset_id","year"]).quantile(0.5)[["datetime_date","depth_m"]].reset_index()
#del dfActiveLayerDepthsFromTempsReduced["year"]
del dfActiveLayerDepthsFromTemps["year"]
del dfActiveLayerDepthsFromTemps["month"]
#dfActiveLayerDepthsFromTempsReduced["subsite_type"] = "temperatures"
dfActiveLayerDepthsFromTemps["subsite_type"] = "temperatures"
dfActiveLayerDepthsFromTemps#Reduced

,datetime_date,depth_m,dataset_id,subsite_type
0,1957-06-15 00:00:00,1.150000,1944,temperatures
1,1953-08-15 12:00:00,2.361905,1944,temperatures
2,1953-09-15 00:00:00,2.311111,1944,temperatures
3,1954-06-15 00:00:00,1.533333,1944,temperatures
4,1954-07-15 12:00:00,1.866667,1944,temperatures
...,...,...,...,...
1038,2015-08-11 00:00:00,2.209302,1964,temperatures
1039,2015-08-12 00:00:00,2.276596,1964,temperatures
1040,2015-08-13 00:00:00,2.298969,1964,temperatures
1041,2006-08-21 00:00:00,1.136986,1964,temperatures


In [71]:
conn.execute("DELETE FROM t_permafrost_depth_data WHERE subsite_type='temperatures'")
#copy_from_stringio(conn.raw_connection(), dfActiveLayerDepthsFromTempsReduced, "t_permafrost_depth_data")
copy_from_stringio(conn.raw_connection(), dfActiveLayerDepthsFromTemps, "t_permafrost_depth_data")

In [72]:
#dfActiveLayerDepthsFromTempsReduced.to_parquet("./mercury/activelayer.from.temperatures.parquet")

## Download Dataset Metadata

Very unfortunately, this was not possible. The Chromium Selenium driver did not seem to like jQueryTables, and the Firefox Gecko driver did not like http unencrypted protocols.

## Postprocessing

### Resampling/re-gridding borehole temperature depth values

As described in the accompaying method notebook [Howto Guide Permafrost Temperatures.ipynb](Howto Guide Permafrost Temperatures.ipynb), we re-map the borehole temperature data to the most frequently used depth values, to allow for comparison between them (note that this does not imply these were comparable, this depends on the experiment set-up and e.g. the surroundings and situation at and near the borehole. But its a starting point.).

In [73]:
# TODO

## Utilities

One of the main reasons that drive me to go through the pain of loading data into SQL datastores is the cleanness of getting the data out. 

### Views

I am using views a lot, I am aware that writing a complicated SQL query is fun, but I rather do various steps, one by one, to create complex queries. This massively reduced the complexity of (python) code, and it significantly reduced data transfer times over networks.


In [74]:
stmt = """CREATE OR REPLACE VIEW v_permafrost_depth_data_annual_stats AS 
SELECT EXTRACT(year FROM datetime_date) as yyyy,dataset_id,subsite_type 
FROM "public"."t_permafrost_depth_data" 
GROUP BY 1,2,3;"""
conn.execute(text(stmt).execution_options(autocommit=True))

In [75]:
stmt = """CREATE OR REPLACE VIEW v_permafrost_depth_data_annual_summary AS 
SELECT COUNT(*),yyyy,subsite_type 
FROM v_permafrost_depth_data_annual_stats 
GROUP BY yyyy,subsite_type;
"""
conn.execute(text(stmt).execution_options(autocommit=True))

In [76]:
#SELECT EXTRACT(month FROM datetime_date) as mm FROM t_permafrost_depth_data WHERE subsite_type='activelayer'

#SELECT EXTRACT(month FROM datetime_date) as mm,COUNT(*) FROM t_permafrost_depth_data WHERE subsite_type='activelayer' GROUP BY 1 ORDER BY 1

In [77]:
stmt = """CREATE OR REPLACE VIEW v_permafrost_depth_data_from_temps AS 
SELECT EXTRACT(year FROM datetime_date) as yyyy,datetime_date,depth_m,dataset_id FROM t_permafrost_depth_data 
WHERE subsite_type='temperatures'"""
conn.execute(text(stmt).execution_options(autocommit=True))

In [78]:
stmt = """CREATE OR REPLACE VIEW v_permafrost_depth_data_from_activelayer AS
SELECT EXTRACT(year FROM datetime_date) as yyyy,datetime_date,depth_m,dataset_id FROM t_permafrost_depth_data WHERE subsite_type='activelayer'"""
conn.execute(text(stmt).execution_options(autocommit=True))

In [79]:
stmt = """CREATE OR REPLACE VIEW v_permafrost_depths_from_temps AS SELECT  yyyy, dataset_id, 
	min(depth_m), 
	percentile_cont(0.1) within group (ORDER BY depth_m) AS q10,
	percentile_cont(0.5) within group (ORDER BY depth_m) AS q50,
	percentile_cont(0.9) within group (ORDER BY depth_m) AS q90,
	max(depth_m) 
FROM v_permafrost_depth_data_from_temps GROUP BY 2,1"""
conn.execute(text(stmt).execution_options(autocommit=True))

stmt = """CREATE OR REPLACE VIEW v_permafrost_depths_from_activelayer AS 
SELECT  yyyy, dataset_id, 
	min(depth_m), 
	percentile_cont(0.1) within group (ORDER BY depth_m) AS q10,
	percentile_cont(0.5) within group (ORDER BY depth_m) AS q50,
	percentile_cont(0.9) within group (ORDER BY depth_m) AS q90,
	max(depth_m) 
FROM v_permafrost_depth_data_from_activelayer GROUP BY 2,1"""
conn.execute(text(stmt).execution_options(autocommit=True))

stmt = """CREATE OR REPLACE VIEW v_permafrost_depths AS
SELECT *,'a' as type FROM v_permafrost_depths_from_activelayer
UNION
SELECT *,'t' as type FROM v_permafrost_depths_from_temps"""
conn.execute(text(stmt).execution_options(autocommit=True))

In [85]:
stmt = """CREATE OR REPLACE VIEW v_permafrost_sites AS
SELECT dataset_id,
	CONCAT(t_permafrost_sites.site_name,'/',
	t_permafrost_catalogue.subsite_name,
	' (',t_permafrost_catalogue.dataset_id,')') AS name
FROM t_permafrost_catalogue
JOIN t_permafrost_sites ON t_permafrost_catalogue.subsite_id=t_permafrost_sites.subsite_id"""
conn.execute(text(stmt).execution_options(autocommit=True))

In [81]:
stmt = """CREATE OR REPLACE VIEW "public"."v_sites" AS
SELECT t_permafrost_catalogue.dataset_id,
    concat(t_permafrost_sites.site_name, '/', t_permafrost_catalogue.subsite_name, ' (', t_permafrost_catalogue.dataset_id, ')') AS name
   FROM (t_permafrost_catalogue
     JOIN t_permafrost_sites ON (((t_permafrost_catalogue.subsite_id)::text = (t_permafrost_sites.subsite_id)::text)));"""
conn.execute(text(stmt).execution_options(autocommit=True))

### Indices

In [82]:
stmt = "CREATE INDEX ON t_permafrost_temperature_data (dataset_id)"
conn.execute(text(stmt).execution_options(autocommit=True))

### Materialized View(s)

This is done to improve query performance

In [83]:
stmt = """CREATE MATERIALIZED VIEW mv_permafrost_depths AS
SELECT v_permafrost_depths_from_activelayer.yyyy,
    v_permafrost_depths_from_activelayer.dataset_id,
    v_permafrost_depths_from_activelayer.min,
    v_permafrost_depths_from_activelayer.q10,
    v_permafrost_depths_from_activelayer.q50,
    v_permafrost_depths_from_activelayer.q90,
    v_permafrost_depths_from_activelayer.max,
    'a'::text AS type
   FROM v_permafrost_depths_from_activelayer
UNION
 SELECT v_permafrost_depths_from_temps.yyyy,
    v_permafrost_depths_from_temps.dataset_id,
    v_permafrost_depths_from_temps.min,
    v_permafrost_depths_from_temps.q10,
    v_permafrost_depths_from_temps.q50,
    v_permafrost_depths_from_temps.q90,
    v_permafrost_depths_from_temps.max,
    't'::text AS type
   FROM v_permafrost_depths_from_temps"""
conn.execute(text(stmt).execution_options(autocommit=True))

In [84]:
stmt = "REFRESH MATERIALIZED VIEW mv_permafrost_depths;"
conn.execute(text(stmt).execution_options(autocommit=True))

### Remaining Columns Comments

In [7]:
stmts = ["COMMENT ON COLUMN t_permafrost_catalogue.aggregation IS 'temperature measurement one of Average,Constant_Over_Interval,Continuous,Sporadic,Unknown'",
"COMMENT ON COLUMN t_permafrost_catalogue.aggregation_frequency IS 'time delta text (such as 12_hours, Daily) of temperature measurements'",
"COMMENT ON COLUMN t_permafrost_catalogue.avg_gap_d IS 'average duration of gap in dataset in [days]'",
"COMMENT ON COLUMN t_permafrost_catalogue.avg_gap_h IS 'average duration of gap in dataset in [hours]'",
"COMMENT ON COLUMN t_permafrost_catalogue.avg_measurements_year IS 'average number of measurements in the dataset [1/year]'",
"COMMENT ON COLUMN t_permafrost_catalogue.dataset_id IS 'Dataset ID, unique for activelayer data only'",
"COMMENT ON COLUMN t_permafrost_catalogue.dropouts_ratio IS 'ratio of dropouts (individual missing measurement records) to numer of records'",
"COMMENT ON COLUMN t_permafrost_catalogue.extension IS 'file name extension of downloaded file'",
"COMMENT ON COLUMN t_permafrost_catalogue.filename IS 'filename of downloaded file'",
"COMMENT ON COLUMN t_permafrost_catalogue.from_dt IS 'datetime/date of first valid datapoint in dataset'",
"COMMENT ON COLUMN t_permafrost_catalogue.gaps_ratio IS 'ratio of gaps (sequences of missing measurement records) to numer of records'",
"COMMENT ON COLUMN t_permafrost_catalogue.index IS 'artefact, ignore, to be removed'",
"COMMENT ON COLUMN t_permafrost_catalogue.instrument IS 'for temperatures, one of Temperature_Interpolation, Thermistor_Automated, Thermistor_Manual, Thermocouple_Automated, Unknown'",
"COMMENT ON COLUMN t_permafrost_catalogue.longest_gap IS 'average duration of gap in dataset in [records]'",
"COMMENT ON COLUMN t_permafrost_catalogue.num_records IS 'number of valid (not-NaN) records in dataset'",
"COMMENT ON COLUMN t_permafrost_catalogue.num_years IS 'number of years covered by dataset, to_dt-from-dt'",
"COMMENT ON COLUMN t_permafrost_catalogue.parameter IS 'parameter of original measurement, one of Ground_Temperature, depth_m'",
"COMMENT ON COLUMN t_permafrost_catalogue.path IS 'pathname of data file'",
"COMMENT ON COLUMN t_permafrost_catalogue.subsite_id IS 'ID of subsite as generated by gtnpdatabase.org'",
"COMMENT ON COLUMN t_permafrost_catalogue.subsite_name IS 'Name of the subsite (usually the hole itself)'",
"COMMENT ON COLUMN t_permafrost_catalogue.subsite_type IS 'one of [activelayer|temperatures], indicating measured source of data'",
"COMMENT ON COLUMN t_permafrost_catalogue.to_dt IS 'datetime/date of last valid datapoint in dataset'",
"COMMENT ON COLUMN t_permafrost_temperature_data.dataset_id IS 'Dataset ID, unique for activelayer data only'",
"COMMENT ON COLUMN t_permafrost_temperature_data.datetime_date IS 'Timestamp of observation'",
"COMMENT ON COLUMN t_permafrost_temperature_data.depth_m IS '(computed) depth of transition into fozen soil'",
"COMMENT ON COLUMN t_permafrost_temperature_data.temperature_degc IS 'measured temperature in [deg Celsius]'",
"COMMENT ON COLUMN v_permafrost_depth_data_annual_stats.dataset_id IS 'Dataset ID, unique for activelayer data only'",
"COMMENT ON COLUMN v_permafrost_depth_data_annual_stats.subsite_type IS 'one of [activelayer|temperatures], indicating measured source of data'",
"COMMENT ON COLUMN v_permafrost_depth_data_annual_stats.yyyy IS 'Year (of an aggregated value)'",
"COMMENT ON COLUMN v_permafrost_depth_data_annual_summary.count IS 'Aggregared number of records for subsite and year'",
"COMMENT ON COLUMN v_permafrost_depth_data_annual_summary.subsite_type IS 'one of [activelayer|temperatures], indicating measured source of data'",
"COMMENT ON COLUMN v_permafrost_depth_data_annual_summary.yyyy IS 'Year (of an aggregated value)'",
"COMMENT ON COLUMN v_permafrost_depth_data_from_activelayer.dataset_id IS 'Dataset ID, unique for activelayer data only'",
"COMMENT ON COLUMN v_permafrost_depth_data_from_activelayer.datetime_date IS 'Timestamp of observation'",
"COMMENT ON COLUMN v_permafrost_depth_data_from_activelayer.yyyy IS 'Year (of an aggregated value)'",
"COMMENT ON COLUMN v_permafrost_depth_data_from_temps.dataset_id IS 'Dataset ID, unique for activelayer data only'",
"COMMENT ON COLUMN v_permafrost_depth_data_from_temps.datetime_date IS 'Timestamp of observation'",
"COMMENT ON COLUMN v_permafrost_depth_data_from_temps.depth_m IS '(measured) depth of transition into fozen soil'",
"COMMENT ON COLUMN v_permafrost_depth_data_from_temps.yyyy IS 'Year (of an aggregated value)'",
"COMMENT ON COLUMN v_permafrost_depths.dataset_id IS 'Dataset ID, unique for activelayer data only'",
"COMMENT ON COLUMN v_permafrost_depths.max IS 'Maximum value of individual depth data records when aggregated'",
"COMMENT ON COLUMN v_permafrost_depths.min IS 'Minimum value of individual depth data records when aggregated'",
"COMMENT ON COLUMN v_permafrost_depths.q10 IS '10% quantile value of individual depth data records when aggregated'",
"COMMENT ON COLUMN v_permafrost_depths.q50 IS '50% quantile value of individual depth data records when aggregated'",
"COMMENT ON COLUMN v_permafrost_depths.q90 IS '90% quantile value of individual depth data records when aggregated'",
"COMMENT ON COLUMN v_permafrost_depths.type IS 'one of [a|t] indicating activelayer or temperature data as source of data point'",
"COMMENT ON COLUMN v_permafrost_depths.yyyy IS 'Year (of an aggregated value)'",
"COMMENT ON COLUMN v_permafrost_depths_from_activelayer.dataset_id IS 'Dataset ID, unique for activelayer data only'",
"COMMENT ON COLUMN v_permafrost_depths_from_activelayer.max IS 'Maximum value of individual depth data records when aggregated'",
"COMMENT ON COLUMN v_permafrost_depths_from_activelayer.min IS 'Minimum value of individual depth data records when aggregated'",
"COMMENT ON COLUMN v_permafrost_depths_from_activelayer.q10 IS '10% quantile value of individual depth data records when aggregated'",
"COMMENT ON COLUMN v_permafrost_depths_from_activelayer.q50 IS '50% quantile value of individual depth data records when aggregated'",
"COMMENT ON COLUMN v_permafrost_depths_from_activelayer.q90 IS '90% quantile value of individual depth data records when aggregated'",
"COMMENT ON COLUMN v_permafrost_depths_from_activelayer.yyyy IS 'Year (of an aggregated value)'",
"COMMENT ON COLUMN v_permafrost_depths_from_temps.dataset_id IS 'Dataset ID, unique for activelayer data only'",
"COMMENT ON COLUMN v_permafrost_depths_from_temps.max IS 'Maximum value of individual depth data records when aggregated'",
"COMMENT ON COLUMN v_permafrost_depths_from_temps.min IS 'Minimum value of individual depth data records when aggregated'",
"COMMENT ON COLUMN v_permafrost_depths_from_temps.q10 IS '10% quantile value of individual depth data records when aggregated'",
"COMMENT ON COLUMN v_permafrost_depths_from_temps.q50 IS '50% quantile value of individual depth data records when aggregated'",
"COMMENT ON COLUMN v_permafrost_depths_from_temps.q90 IS '90% quantile value of individual depth data records when aggregated'",
"COMMENT ON COLUMN v_permafrost_depths_from_temps.yyyy IS 'Year (of an aggregated value)'",
"COMMENT ON COLUMN v_sites.dataset_id IS 'Dataset ID, unique for activelayer data only'",
"COMMENT ON COLUMN v_sites.name IS 'Site name'"]

for stmt in stmts:
    conn.execute(text(stmt).execution_options(autocommit=True))